# Load dependencies

In [2]:
import cobra
import libsbml
import lxml

models_directory = '/media/sf_Shared/Systems_biology/Metabolic_models/'
OB3b_directory = '/media/sf_Shared/GEM_OB3b/'
memote_directory = '/home/ensakz/Desktop/memote_m_trichosporium/'
fastas_directory = '/media/sf_Shared/Systems_biology/Fastas_and_annotations/'
inparanoid_directory = '/media/sf_Shared/Systems_biology/InParanoid_runs/'
draft_gems_dir_8 = 'draft_gems/8.OB3b_draft_gem_during_pathway_manual_curation/'

In [3]:
iMsOB3b = cobra.io.load_json_model(memote_directory + draft_gems_dir_8 + "bigg_OB3b_13_01_2020.json")

In [3]:
model_Bath = cobra.io.load_json_model(models_directory + "iMcBath.json")

In [4]:
model_OB3b_old = cobra.io.load_json_model(memote_directory + draft_gems_dir_8 + "iMsOB3b_11_14_2019.json")

In [5]:
model_universal = cobra.io.load_json_model(models_directory + 'universal_model.json')

In [81]:
model_AM1 = cobra.io.read_sbml_model(models_directory + 'iRP911_v2016_MextorquensAM1.xml')

# Create list of Methylosinus trichosporium OB3b locus tags not present in OB3b GEM

Loading BioPython's SeqIO library for fasta file manipulation:

In [36]:
model = iMsOB3b

In [37]:
#script for renaming record description in Refseq or Genbank files

#importing Biopythom SeqIO library
from Bio import SeqIO

#loading multiple gene elements from a fasta file
records = list(SeqIO.parse(fastas_directory + "OB3b/New_2017_assembly/GenBank_ASM275265v1_translated_cds.faa", 
                           "fasta"))

Create OB3b_locus_tags_and_their_description dictionary:

In [38]:
OB3b_locus_tags_and_their_description = {}
for record in records:
    record_description = record.description.split("[") #some of elements don't have WP_ ids, only locus tags
    for element in record_description:
        if "locus_tag=" in element:
            key_element=element[10:-2]
        elif "protein=" in element:
            value_element=element[8:-2]
    OB3b_locus_tags_and_their_description[str(key_element)] = str(value_element)
print(len(OB3b_locus_tags_and_their_description))

4740


Write OB3b_locus_tags_and_their_description to txt file:

In [39]:
import json
with open(memote_directory + "gem_reconstruction_supplementary_materials/" +\
          "OB3b_locus_tags_and_their_description.txt", "w") as file:
    file.write(json.dumps(OB3b_locus_tags_and_their_description))

Serialization using Pickle fails:
<br> Source: https://stackoverflow.com/questions/36965507/writing-a-dictionary-to-a-text-file

In [40]:
import _pickle as pickle
with open(memote_directory + "gem_reconstruction_supplementary_materials/" +\
          "OB3b_locus_tags_and_their_description_parallel.txt", 'wb') as file:
     file.write(pickle.dumps(OB3b_locus_tags_and_their_description)) 

Check how much Genbank genome locus tags are present in OB3b GEM:

In [41]:
OB3b_locus_tags_present_in_GEM = set()
for locus_tag in OB3b_locus_tags_and_their_description:
    try:
        if iMsOB3b.genes.get_by_id(locus_tag) in iMsOB3b.genes:
            OB3b_locus_tags_present_in_GEM.add(locus_tag)
    except:
        pass
OB3b_locus_tags_present_in_GEM=list(OB3b_locus_tags_present_in_GEM)
print(len(OB3b_locus_tags_present_in_GEM))

537


Deleting "CQW"-related genes present in OB3b GEM from OB3b_locus_tags_and_their_description:

In [42]:
OB3b_locus_tags_present_in_GEM = set()
for locus_tag in OB3b_locus_tags_and_their_description:
    try:
        if model.genes.get_by_id(locus_tag) in model.genes:
            OB3b_locus_tags_present_in_GEM.add(locus_tag)
    except:
        pass
OB3b_locus_tags_present_in_GEM=list(OB3b_locus_tags_present_in_GEM)

for locus_tag in OB3b_locus_tags_present_in_GEM:
    del OB3b_locus_tags_and_their_description[locus_tag]
    
len(OB3b_locus_tags_and_their_description)

4203

Deleting hypothetical proteins from OB3b_locus_tags_and_their_description as they are useless for GEM reconstruction manual curation:

In [50]:
for locus_tag in list(OB3b_locus_tags_and_their_description):
    if OB3b_locus_tags_and_their_description[locus_tag] == "hypothetical protein":
        del OB3b_locus_tags_and_their_description[locus_tag]

print("OB3b_locus_tags_and_their_description length after deletion of hypothetical proteins is",\
      len(OB3b_locus_tags_and_their_description))        

OB3b_locus_tags_and_their_description length after deletion of hypothetical proteins is 2880


In [13]:
OB3b_locus_tags_and_their_description

{'CQW49_00005': 'chromosomal replication initiator protein DnaA',
 'CQW49_00010': 'glycosyl transferase',
 'CQW49_00020': 'peptidase M3',
 'CQW49_00025': 'flavin reductase family protein',
 'CQW49_00030': 'bifunctional transcriptional activator/DNA repair protein Ada',
 'CQW49_00040': 'DUF202 domain-containing protein',
 'CQW49_00065': 'toxin-antitoxin system',
 'CQW49_00070': 'L-histidine N(alpha)-methyltransferase',
 'CQW49_00075': 'ergothioneine biosynthesis protein EgtB',
 'CQW49_00080': 'acyl carrier protein',
 'CQW49_00085': 'L,D-transpeptidase',
 'CQW49_00095': 'IS630 family transposase',
 'CQW49_00100': 'acyltransferase',
 'CQW49_00105': 'oxidoreductase',
 'CQW49_00110': 'methyltransferase',
 'CQW49_00120': 'GGDEF domain-containing protein',
 'CQW49_00125': 'CDP-alcohol phosphatidyltransferase',
 'CQW49_00130': 'sugar transporter',
 'CQW49_00140': 'NAD-dependent dehydratase',
 'CQW49_00150': 'glycosyltransferase family 1 protein',
 'CQW49_00190': 'amino acid ABC transporter sub

Saving model copy before moving further in case of any mistake in the code:

In [182]:
model_copy=model.copy()

Checking whether remaining 2880 locus tags from OB3b_locus_tags_and_their_description are present in KEGG  pathway database (and supposedly have EC numbers). If they are not present in KEGG pathway database, then these locus tags are deleted by the script given below. This is possible because KEGG pathway is biochemical metabolic pathway database (opinion as of 15/10/2019).

In [14]:
import bioservices
#trying bioservices
#organism code cah

from bioservices.kegg import KEGG
k = KEGG() #you have to give the alias (psevdonim) for the KEGG database
#you specify organism name, in this case, cah, belongs to Methylosinus trichosporium

k.organism = "mtw"

In [185]:
for locus_tag in list(OB3b_locus_tags_and_their_description):
    try:
        if k.get_pathway_by_gene(locus_tag, "mtw") is None:
            del OB3b_locus_tags_and_their_description[locus_tag]
    except:
        pass

Locus tags are not filtered properly using k.get_pathway_by_gene(locus_tag, "mtw")) is None option.
<br> Trying different locus tag option:
<br> Below is a trash to understand how to apply another solution besides None

In [90]:
if len(k.get_pathway_by_gene('CQW49_14475', "mtw"))>0:
    print('None')
else:
    print('Not none')

None


In [48]:
len(k.get_pathway_by_gene('CQW49_14475', "mtw")) >0

True

In [49]:
checking_reactions = ['CQW49_14475', 'CQW49_00015']
for locus_tag in checking_reactions:
    try:
        if len(k.get_pathway_by_gene(locus_tag, "mtw"))>0:
            print(locus_tag, 'True')
        else:
            print(locus_tag, 'Not true')
    except:
        print(locus_tag, 'None')

CQW49_14475 True
CQW49_00015 None


In [51]:

check_reactions = ['CQW49_14475', 'CQW49_00015']
for reaction in check_reactions:
    try:
        if len(k.get_pathway_by_gene(reaction, "mtw"))>0:
            print(reaction, 'None')
        else:
            print(reaction, 'Not none')
    except:
        print(reaction, 'exception')

CQW49_14475 None
CQW49_00015 exception


Trying second option (sorting relevant locus tags by length):

In [15]:
for locus_tag in list(OB3b_locus_tags_and_their_description):
    try:
        if len(k.get_pathway_by_gene(locus_tag, "mtw")) > 0:
            pass
        else:
            del OB3b_locus_tags_and_their_description[locus_tag]
    except:
            del OB3b_locus_tags_and_their_description[locus_tag]

In [57]:
len(OB3b_locus_tags_and_their_description)

2880

In [54]:
for locus_tag in list(OB3b_locus_tags_and_their_description):
    try:
        if len(k.get_pathway_by_gene(locus_tag, "mtw")) > 0:
            print(locus_tag, k.get_pathway_by_gene(locus_tag, "mtw"))
        else:
            pass
    except:
            pass

CQW49_00005 {'mtw02020': 'Two-component system'}
CQW49_00130 {'mtw02020': 'Two-component system'}
CQW49_00190 {'mtw02010': 'ABC transporters'}
CQW49_00200 {'mtw03010': 'Ribosome'}
CQW49_00370 {'mtw03070': 'Bacterial secretion system'}
CQW49_00375 {'mtw03070': 'Bacterial secretion system'}
CQW49_00380 {'mtw03070': 'Bacterial secretion system'}
CQW49_00385 {'mtw03070': 'Bacterial secretion system'}
CQW49_00390 {'mtw03070': 'Bacterial secretion system'}
CQW49_00400 {'mtw03070': 'Bacterial secretion system'}
CQW49_00415 {'mtw03070': 'Bacterial secretion system'}
CQW49_00420 {'mtw03070': 'Bacterial secretion system'}
CQW49_00425 {'mtw03070': 'Bacterial secretion system'}
CQW49_00500 {'mtw03410': 'Base excision repair'}
CQW49_00515 {'mtw00190': 'Oxidative phosphorylation', 'mtw01100': 'Metabolic pathways'}
CQW49_00520 {'mtw00190': 'Oxidative phosphorylation', 'mtw01100': 'Metabolic pathways'}
CQW49_00535 {'mtw00190': 'Oxidative phosphorylation', 'mtw01100': 'Metabolic pathways'}
CQW49_00540 

CQW49_14460 {'mtw00680': 'Methane metabolism', 'mtw01120': 'Microbial metabolism in diverse environments'}
CQW49_14465 {'mtw00680': 'Methane metabolism', 'mtw01120': 'Microbial metabolism in diverse environments'}
CQW49_14470 {'mtw00680': 'Methane metabolism', 'mtw01120': 'Microbial metabolism in diverse environments'}
CQW49_14475 {'mtw00680': 'Methane metabolism', 'mtw01120': 'Microbial metabolism in diverse environments'}
CQW49_14480 {'mtw00680': 'Methane metabolism', 'mtw01120': 'Microbial metabolism in diverse environments'}
CQW49_14495 {'mtw02020': 'Two-component system'}
CQW49_14840 {'mtw02020': 'Two-component system'}
CQW49_14845 {'mtw02020': 'Two-component system'}
CQW49_14880 {'mtw02020': 'Two-component system', 'mtw02024': 'Quorum sensing'}
CQW49_17050 {'mtw02020': 'Two-component system'}
CQW49_17055 {'mtw02020': 'Two-component system'}
CQW49_17075 {'mtw00620': 'Pyruvate metabolism', 'mtw01100': 'Metabolic pathways'}
CQW49_17085 {'mtw02010': 'ABC transporters'}
CQW49_17090 {'

In [55]:
for locus_tag in list(OB3b_locus_tags_and_their_description):
    try:
        if len(k.get_pathway_by_gene(locus_tag, "mtw")) > 0:
            pass
        else:
            pass
    except:
            print(locus_tag, k.get_pathway_by_gene(locus_tag, "mtw"))

CQW49_00010 None
CQW49_00020 None
CQW49_00025 None
CQW49_00030 None
CQW49_00040 None
CQW49_00065 None
CQW49_00070 None
CQW49_00075 None
CQW49_00080 None
CQW49_00085 None
CQW49_00095 None
CQW49_00100 None
CQW49_00105 None
CQW49_00110 None
CQW49_00120 None
CQW49_00125 None
CQW49_00140 None
CQW49_00150 None
CQW49_00195 None
CQW49_00210 None
CQW49_00215 None
CQW49_00220 None
CQW49_00225 None
CQW49_00235 None
CQW49_00245 None
CQW49_00255 None
CQW49_00260 None
CQW49_00275 None
CQW49_00325 None
CQW49_00330 None
CQW49_00340 None
CQW49_00365 None
CQW49_00405 None
CQW49_00440 None
CQW49_00460 None
CQW49_00465 None
CQW49_00490 None
CQW49_00495 None
CQW49_00510 None
CQW49_00525 None
CQW49_00565 None
CQW49_00580 None
CQW49_00585 None
CQW49_00590 None
CQW49_00595 None
CQW49_00605 None
CQW49_00615 None
CQW49_00620 None
CQW49_00625 None
CQW49_00630 None
CQW49_00645 None
CQW49_00650 None
CQW49_00655 None
CQW49_00665 None
CQW49_00685 None
CQW49_00690 None
CQW49_00700 None
CQW49_00705 None
CQW49_00710 No

CQW49_09245 None
CQW49_09250 None
CQW49_09255 None
CQW49_09260 None
CQW49_09270 None
CQW49_09275 None
CQW49_09280 None
CQW49_09290 None
CQW49_09295 None
CQW49_09300 None
CQW49_09305 None
CQW49_09325 None
CQW49_09330 None
CQW49_09335 None
CQW49_09340 None
CQW49_09345 None
CQW49_09350 None
CQW49_09355 None
CQW49_09365 None
CQW49_09370 None
CQW49_09375 None
CQW49_09380 None
CQW49_09395 None
CQW49_09400 None
CQW49_09405 None
CQW49_09415 None
CQW49_09420 None
CQW49_09425 None
CQW49_09440 None
CQW49_09450 None
CQW49_09455 None
CQW49_09465 None
CQW49_09470 None
CQW49_09485 None
CQW49_09500 None
CQW49_09510 None
CQW49_09550 None
CQW49_09570 None
CQW49_09575 None
CQW49_09595 None
CQW49_09610 None
CQW49_09615 None
CQW49_09675 None
CQW49_09685 None
CQW49_09705 None
CQW49_09710 None
CQW49_09725 None
CQW49_09730 None
CQW49_09740 None
CQW49_09745 None
CQW49_09750 None
CQW49_09755 None
CQW49_09760 None
CQW49_09765 None
CQW49_09770 None
CQW49_09775 None
CQW49_09780 None
CQW49_09785 None
CQW49_09790 No

CQW49_18015 None
CQW49_18025 None
CQW49_18045 None
CQW49_18050 None
CQW49_18055 None
CQW49_18070 None
CQW49_18075 None
CQW49_18080 None
CQW49_18085 None
CQW49_18095 None
CQW49_18105 None
CQW49_18110 None
CQW49_18115 None
CQW49_18125 None
CQW49_18145 None
CQW49_18150 None
CQW49_18155 None
CQW49_18175 None
CQW49_18180 None
CQW49_18185 None
CQW49_18200 None
CQW49_18205 None
CQW49_18210 None
CQW49_18215 None
CQW49_18220 None
CQW49_18225 None
CQW49_18240 None
CQW49_18245 None
CQW49_18265 None
CQW49_18270 None
CQW49_18275 None
CQW49_18320 None
CQW49_18325 None
CQW49_18330 None
CQW49_18335 None
CQW49_18340 None
CQW49_18350 None
CQW49_18360 None
CQW49_18365 None
CQW49_18380 None
CQW49_18415 None
CQW49_18435 None
CQW49_18440 None
CQW49_18445 {'mtw03410': 'Base excision repair'}
CQW49_18490 None
CQW49_18495 None
CQW49_18500 None
CQW49_18505 None
CQW49_18510 None
CQW49_18515 None
CQW49_18520 None
CQW49_18525 None
CQW49_18530 None
CQW49_18535 None
CQW49_18540 None
CQW49_18545 None
CQW49_18560 None

In [23]:
len(OB3b_locus_tags_and_their_description)

315

Save the OB3b_locus_tags_and_their_description temporal file with pickle:

In [17]:
# write python dict to a file
with open(memote_directory + "gem_reconstruction_supplementary_materials/AM1_based_GEM/" +\
          'temporal_more_than_0.pkl', 'wb') as output:
    pickle.dump(OB3b_locus_tags_and_their_description, output)

In [20]:
for locus_tag in OB3b_locus_tags_and_their_description:
    print(locus_tag, k.get_pathway_by_gene(locus_tag, "mtw"))

CQW49_00005 {'mtw02020': 'Two-component system'}
CQW49_00130 {'mtw02020': 'Two-component system'}
CQW49_00190 {'mtw02010': 'ABC transporters'}
CQW49_00200 {'mtw03010': 'Ribosome'}
CQW49_00370 {'mtw03070': 'Bacterial secretion system'}
CQW49_00375 {'mtw03070': 'Bacterial secretion system'}
CQW49_00380 {'mtw03070': 'Bacterial secretion system'}
CQW49_00385 {'mtw03070': 'Bacterial secretion system'}
CQW49_00390 {'mtw03070': 'Bacterial secretion system'}
CQW49_00400 {'mtw03070': 'Bacterial secretion system'}
CQW49_00415 {'mtw03070': 'Bacterial secretion system'}
CQW49_00420 {'mtw03070': 'Bacterial secretion system'}
CQW49_00425 {'mtw03070': 'Bacterial secretion system'}
CQW49_00500 {'mtw03410': 'Base excision repair'}
CQW49_00515 {'mtw00190': 'Oxidative phosphorylation', 'mtw01100': 'Metabolic pathways'}
CQW49_00520 {'mtw00190': 'Oxidative phosphorylation', 'mtw01100': 'Metabolic pathways'}
CQW49_00535 {'mtw00190': 'Oxidative phosphorylation', 'mtw01100': 'Metabolic pathways'}
CQW49_00540 

CQW49_14130 {'mtw02024': 'Quorum sensing'}
CQW49_14135 {'mtw02024': 'Quorum sensing'}
CQW49_14310 {'mtw00520': 'Amino sugar and nucleotide sugar metabolism', 'mtw01100': 'Metabolic pathways'}
CQW49_14315 {'mtw00520': 'Amino sugar and nucleotide sugar metabolism', 'mtw01100': 'Metabolic pathways'}
CQW49_14320 {'mtw03010': 'Ribosome'}
CQW49_14405 {'mtw00920': 'Sulfur metabolism', 'mtw01100': 'Metabolic pathways', 'mtw01120': 'Microbial metabolism in diverse environments'}
CQW49_14430 {'mtw00680': 'Methane metabolism', 'mtw01120': 'Microbial metabolism in diverse environments'}
CQW49_14435 {'mtw00680': 'Methane metabolism', 'mtw01120': 'Microbial metabolism in diverse environments'}
CQW49_14460 {'mtw00680': 'Methane metabolism', 'mtw01120': 'Microbial metabolism in diverse environments'}
CQW49_14465 {'mtw00680': 'Methane metabolism', 'mtw01120': 'Microbial metabolism in diverse environments'}
CQW49_14470 {'mtw00680': 'Methane metabolism', 'mtw01120': 'Microbial metabolism in diverse envir

In [24]:
OB3b_locus_tags_and_their_description

{'CQW49_00005': 'chromosomal replication initiator protein DnaA',
 'CQW49_00130': 'sugar transporter',
 'CQW49_00190': 'amino acid ABC transporter substrate-bindnig protein',
 'CQW49_00200': '30S ribosomal protein S21',
 'CQW49_00370': 'prepilin-type cleavage/methylation domain-containing protein',
 'CQW49_00375': 'type II secretion system protein',
 'CQW49_00380': 'type II secretion system protein GspH',
 'CQW49_00385': 'type II secretion system F family protein',
 'CQW49_00390': 'secretion system protein E',
 'CQW49_00400': 'fimbrial assembly protein',
 'CQW49_00415': 'type II secretion system protein GspD',
 'CQW49_00420': 'general secretion pathway protein GspK',
 'CQW49_00425': 'type II secretion system protein GspG',
 'CQW49_00500': 'uracil-DNA glycosylase',
 'CQW49_00515': 'F0F1 ATP synthase subunit beta',
 'CQW49_00520': 'F0F1 ATP synthase subunit epsilon',
 'CQW49_00535': 'ATP synthase F0 subunit A',
 'CQW49_00540': 'ATP synthase F0 subunit C',
 'CQW49_00545': 'F0F1 ATP syntha

In [142]:
len(OB3b_locus_tags_and_their_description)

341

In [25]:
check_reactions = ['CQW49_19995', 'CQW49_20020', 'CQW49_23400']
for locus_tag in check_reactions:
    print(k.get_pathway_by_gene(locus_tag, "mtw"))

{'mtw03440': 'Homologous recombination'}
{'mtw00630': 'Glyoxylate and dicarboxylate metabolism', 'mtw01100': 'Metabolic pathways'}
{'mtw02020': 'Two-component system'}


Printing length and elements of OB3b_locus_tags_and_their_description. The number of locus tags that can be potentially included in genome-scale metabolic reconstruction is 341.

In [140]:
print(len(OB3b_locus_tags_and_their_description))
OB3b_locus_tags_and_their_description

341


{'CQW49_00005': 'chromosomal replication initiator protein DnaA',
 'CQW49_00130': 'sugar transporter',
 'CQW49_00190': 'amino acid ABC transporter substrate-bindnig protein',
 'CQW49_00200': '30S ribosomal protein S21',
 'CQW49_00370': 'prepilin-type cleavage/methylation domain-containing protein',
 'CQW49_00375': 'type II secretion system protein',
 'CQW49_00380': 'type II secretion system protein GspH',
 'CQW49_00385': 'type II secretion system F family protein',
 'CQW49_00390': 'secretion system protein E',
 'CQW49_00400': 'fimbrial assembly protein',
 'CQW49_00415': 'type II secretion system protein GspD',
 'CQW49_00420': 'general secretion pathway protein GspK',
 'CQW49_00425': 'type II secretion system protein GspG',
 'CQW49_00500': 'uracil-DNA glycosylase',
 'CQW49_00515': 'F0F1 ATP synthase subunit beta',
 'CQW49_00520': 'F0F1 ATP synthase subunit epsilon',
 'CQW49_00535': 'ATP synthase F0 subunit A',
 'CQW49_00540': 'ATP synthase F0 subunit C',
 'CQW49_00545': 'F0F1 ATP syntha

Writing filtered OB3b_locus_tags_and_their_description to memote/gem_reconstruction_supplementary_materials directory in pkl and txt formats. The pkl format is required to always have access to the data using Python while the txt format is required for easy data access using txt editor:

In [188]:
# write python dict to a file
with open(memote_directory + "gem_reconstruction_supplementary_materials/AM1_based_GEM/" +\
          'OB3b_locus_tags_and_their_description_not_included_in_automatic_GEMs.pkl', 'wb') as output:
    pickle.dump(OB3b_locus_tags_and_their_description, output)
    
import json
with open(memote_directory + "gem_reconstruction_supplementary_materials/AM1_based_GEM/" +\
          "OB3b_locus_tags_and_their_description_not_included_in_automatic_GEMs.txt", "w") as file:
    file.write(json.dumps(OB3b_locus_tags_and_their_description))

Easy access to OB3b_locus_tags_and_their_description_not_included_in_automatic_GEMs.pkl:

In [214]:
# read python dict back from the file
pkl_file = open(memote_directory + "gem_reconstruction_supplementary_materials/AM1_based_GEM/" +\
                'OB3b_locus_tags_and_their_description_not_included_in_automatic_GEMs.pkl', 'rb')
OB3b_locus_tags_and_their_description = pickle.load(pkl_file)
pkl_file.close()

print (OB3b_locus_tags_and_their_description)

{'CQW49_00005': 'chromosomal replication initiator protein DnaA', 'CQW49_00130': 'sugar transporter', 'CQW49_00190': 'amino acid ABC transporter substrate-bindnig protein', 'CQW49_00200': '30S ribosomal protein S21', 'CQW49_00370': 'prepilin-type cleavage/methylation domain-containing protein', 'CQW49_00375': 'type II secretion system protein', 'CQW49_00380': 'type II secretion system protein GspH', 'CQW49_00385': 'type II secretion system F family protein', 'CQW49_00390': 'secretion system protein E', 'CQW49_00400': 'fimbrial assembly protein', 'CQW49_00415': 'type II secretion system protein GspD', 'CQW49_00420': 'general secretion pathway protein GspK', 'CQW49_00425': 'type II secretion system protein GspG', 'CQW49_00500': 'uracil-DNA glycosylase', 'CQW49_00515': 'F0F1 ATP synthase subunit beta', 'CQW49_00520': 'F0F1 ATP synthase subunit epsilon', 'CQW49_00535': 'ATP synthase F0 subunit A', 'CQW49_00540': 'ATP synthase F0 subunit C', 'CQW49_00545': 'F0F1 ATP synthase subunit B', 'CQ

Leave only pathways that are directly related to metabolism:
Here is the list of pathways not related to metabolism directly (starts from genetic information processing section) --> https://www.genome.jp/kegg-bin/show_organism?menu_type=pathway_maps&org=mtw:

In [27]:
kegg_non_metabolic_pathways = ['03020', '03010', '00970', '03060', '04122', '03018', '03030', '03410', 
                               '03420','03430', '03440', '03450', '02010', '02060', '03070', '02020', 
                               '02024', '02030', '02040', '01501', '01502', '01503']

In [28]:
for locus_tag in list(OB3b_locus_tags_and_their_description):
    for non_metabolic_pathway in kegg_non_metabolic_pathways:
        try:
            if 'mtw' + non_metabolic_pathway in k.get_pathway_by_gene(locus_tag, "mtw"):
                del OB3b_locus_tags_and_their_description[locus_tag]
        except:
            pass

In [31]:
OB3b_locus_tags_and_their_description

{'CQW49_00500': 'uracil-DNA glycosylase',
 'CQW49_00515': 'F0F1 ATP synthase subunit beta',
 'CQW49_00520': 'F0F1 ATP synthase subunit epsilon',
 'CQW49_00535': 'ATP synthase F0 subunit A',
 'CQW49_00540': 'ATP synthase F0 subunit C',
 'CQW49_00545': 'F0F1 ATP synthase subunit B',
 'CQW49_00550': 'F0F1 ATP synthase subunit alpha',
 'CQW49_00555': 'H(+)-transporting ATPase',
 'CQW49_00610': 'exodeoxyribonuclease VII large subunit',
 'CQW49_00660': 'recombination protein RecR',
 'CQW49_00670': 'DNA polymerase III subunit gamma/tau',
 'CQW49_00695': 'glutathione S-transferase family protein',
 'CQW49_00740': 'nucleotide sugar dehydrogenase',
 'CQW49_00750': 'glycosyl transferase family 1',
 'CQW49_00915': 'molybdenum cofactor guanylyltransferase MobA',
 'CQW49_00955': 'histidinol-phosphatase',
 'CQW49_01035': 'phosphoenolpyruvate--protein phosphotransferase',
 'CQW49_01055': 'NAD-dependent DNA ligase LigA',
 'CQW49_01105': 'chorismate mutase',
 'CQW49_01170': '2-dehydropantoate 2-reductas

In [231]:
# write python dict to a file
with open(memote_directory + "gem_reconstruction_supplementary_materials/AM1_based_GEM/" +\
          'OB3b_locus_tags_and_their_description_not_included_in_automatic_GEMs.pkl', 'wb') as output:
    pickle.dump(OB3b_locus_tags_and_their_description, output)
    
import json
with open(memote_directory + "gem_reconstruction_supplementary_materials/AM1_based_GEM/" +\
          "OB3b_locus_tags_and_their_description_not_included_in_automatic_GEMs.txt", "w") as file:
    file.write(json.dumps(OB3b_locus_tags_and_their_description))

In [235]:
# read python dict back from the file
pkl_file = open(memote_directory + "gem_reconstruction_supplementary_materials/AM1_based_GEM/" +\
                'OB3b_locus_tags_and_their_description_not_included_in_automatic_GEMs.pkl', 'rb')
OB3b_locus_tags_and_their_description = pickle.load(pkl_file)
pkl_file.close()

print (len(OB3b_locus_tags_and_their_description), '\n', OB3b_locus_tags_and_their_description)

164 
 {'CQW49_00415': 'type II secretion system protein GspD', 'CQW49_00425': 'type II secretion system protein GspG', 'CQW49_00500': 'uracil-DNA glycosylase', 'CQW49_00515': 'F0F1 ATP synthase subunit beta', 'CQW49_00520': 'F0F1 ATP synthase subunit epsilon', 'CQW49_00535': 'ATP synthase F0 subunit A', 'CQW49_00540': 'ATP synthase F0 subunit C', 'CQW49_00545': 'F0F1 ATP synthase subunit B', 'CQW49_00550': 'F0F1 ATP synthase subunit alpha', 'CQW49_00555': 'H(+)-transporting ATPase', 'CQW49_00610': 'exodeoxyribonuclease VII large subunit', 'CQW49_00660': 'recombination protein RecR', 'CQW49_00670': 'DNA polymerase III subunit gamma/tau', 'CQW49_00695': 'glutathione S-transferase family protein', 'CQW49_00740': 'nucleotide sugar dehydrogenase', 'CQW49_00750': 'glycosyl transferase family 1', 'CQW49_00915': 'molybdenum cofactor guanylyltransferase MobA', 'CQW49_00955': 'histidinol-phosphatase', 'CQW49_01035': 'phosphoenolpyruvate--protein phosphotransferase', 'CQW49_01055': 'NAD-dependent

# Add reactions based on the files:
2 files to look for are reactions are OB3b_locus_tags_and_their_description_not_included_in_automatic_GEMs.pkl and OB3b_locus_tags_EC_number_no_KEGG_subsystems_filtered.pkl
<br>**Note:** Ignore genes which have unspecified metabolites in reactions

In [534]:
for locus_tag in OB3b_locus_tags_and_their_description:
    gene_entry = genes["mtw:"+locus_tag]
    print(locus_tag, gene_entry.orthology)

CQW49_00515 K02112  F-type H+/Na+-transporting ATPase subunit beta [EC:7.1.2.2 7.2.2.1]
CQW49_00550 K02111  F-type H+/Na+-transporting ATPase subunit alpha [EC:7.1.2.2 7.2.2.1]
CQW49_00695 K00799  glutathione S-transferase [EC:2.5.1.18]
CQW49_00740 K13015  UDP-N-acetyl-D-glucosamine dehydrogenase [EC:1.1.1.136]
CQW49_00750 K13668  phosphatidyl-myo-inositol dimannoside synthase [EC:2.4.1.346]
CQW49_00915 K03752  molybdenum cofactor guanylyltransferase [EC:2.7.7.77]
CQW49_00955 K01092  myo-inositol-1(or 4)-monophosphatase [EC:3.1.3.25]
CQW49_01105 K04782  isochorismate pyruvate lyase [EC:4.2.99.21]
CQW49_01170 K00077  2-dehydropantoate 2-reductase [EC:1.1.1.169]
CQW49_01220 K10944  methane/ammonia monooxygenase subunit A [EC:1.14.18.3 1.14.99.39]
CQW49_01315 K07258  serine-type D-Ala-D-Ala carboxypeptidase (penicillin-binding protein 5/6) [EC:3.4.16.4]
CQW49_01360 K00940  nucleoside-diphosphate kinase [EC:2.7.4.6]
CQW49_01395 K11717  cysteine desulfurase / selenocysteine lyase [EC:2.8.1.

In [561]:
len(OB3b_locus_tags_and_their_description)

531

In [540]:
for locus_tag in temporal_file:
    gene_entry = genes["mtw:"+locus_tag]
    print(locus_tag, gene_entry.orthology)

CQW49_00140 K00091  dihydroflavonol-4-reductase [EC:1.1.1.219]
CQW49_01090 K07447  putative holliday junction resolvase [EC:3.1.-.-]
CQW49_01150 K03827  putative acetyltransferase [EC:2.3.1.-]
CQW49_01200 K00428  cytochrome c peroxidase [EC:1.11.1.5]
CQW49_01205 K00428  cytochrome c peroxidase [EC:1.11.1.5]
CQW49_01435 K09001  anhydro-N-acetylmuramic acid kinase [EC:2.7.1.170]
CQW49_01520 K03824  putative acetyltransferase [EC:2.3.1.-]
CQW49_01535 K11209  GSH-dependent disulfide-bond oxidoreductase [EC:1.8.4.-]
CQW49_01600 K03424  TatD DNase family protein [EC:3.1.21.-]
CQW49_01720 K01286  D-alanyl-D-alanine carboxypeptidase [EC:3.4.16.4]
CQW49_01820 K04090  indolepyruvate ferredoxin oxidoreductase [EC:1.2.7.8]
CQW49_02170 K03564  peroxiredoxin Q/BCP [EC:1.11.1.15]
CQW49_02185 K00573  protein-L-isoaspartate(D-aspartate) O-methyltransferase [EC:2.1.1.77]
CQW49_03190 K03579  ATP-dependent helicase HrpB [EC:3.6.4.13]
CQW49_03350 K06137  pyrroloquinoline-quinone synthase [EC:1.3.3.11]
CQW4

In [559]:
len(temporal_file)

65

## Try manual way to add reactions

### ATPase genes: CQW49_00515, CQW49_00550

In [491]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("ATP", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

ATPM ATP maintenance requirement
CU2abcpp copper exporting ATPase
Kabcpp potassium transporting ATPase
PRATPP phosphoribosyl ATP pyrophosphatase
ATPS4rpp ATP synthase
ATPPRTr ATP phosphoribosyltransferase
NTPP5 deoxynucleoside triphosphate pyrophosphohydrolase  dATP 
NTPP6 nucleoside triphosphate pyrophosphohydrolase  ATP 


In [535]:
iMsOB3b.reactions.ATPS4rpp.gene_reaction_rule = 'CQW49_19220 and CQW49_00300 and CQW49_19240 and CQW49_19235 and CQW49_00305 and CQW49_19230 and CQW49_19225 and CQW49_00310 and CQW49_00290 and CQW49_00295 and CQW49_00515 and CQW49_00550'

### CQW49_00695 --> unspecified, vague metabolites
https://www.genome.jp/dbget-bin/www_bget?ec:2.5.1.18

In [507]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("glutathione", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

GTHS glutathione synthetase
LGTHL lactoylglutathione lyase
GTHPp glutathione peroxidase  periplasmic 
GTHOr glutathione reductase
GLYOX S lactoylglutathione hydrolase


In [502]:
for reaction in model_OB3b_old.reactions:
    if re.search("glutathione
                 ", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

DSBGGT DsbG:glutathione thiotransferase


In [503]:
model_OB3b_old.reactions.DSBGGT

Reaction identifier,DSBGGT
Name,DsbG:glutathione thiotransferase
Memory address,0x07f32a737b780
Stoichiometry,dsbgox_p + 2.0 gthrd_p --> dsbgrd_p + gthox_p Periplasmic disulfide isomerase/thiol-disulphide oxidase (oxidized) + 2.0 Reduced glutathione --> Periplasmic disulfide isomerase/thiol-disulphide oxidase (reduced) + Oxidized glutathione
GPR,CQW49_23230
Lower bound,0.0
Upper bound,1000.0


### CQW49_00740

In [511]:
iMsOB3b.add_reaction(model_universal.reactions.UDPACGA6D)

In [513]:
iMsOB3b.reactions.UDPACGA6D.gene_reaction_rule = "CQW49_00740"

## Semiautomatic way of adding reactions to GEM:

List of models from which I can take reactions based on gene reaction rules:
- old OB3b Carveme-based model


List of models from which I take reactions based on reaction name:
- old AM1 model (this is not possible as ortholog analysis for old AM1 model has been already done)

Reload pickle again:

In [578]:
import pickle
pkl_file = open(memote_directory + "gem_reconstruction_supplementary_materials/AM1_based_GEM/" +\
                'OB3b_locus_tags_and_their_description_not_included_in_automatic_GEMs.pkl', 'rb')
OB3b_locus_tags_and_their_description = pickle.load(pkl_file)
pkl_file.close()

print (len(OB3b_locus_tags_and_their_description))

187


In [579]:
import pickle
pkl_file = open(memote_directory + "gem_reconstruction_supplementary_materials/AM1_based_GEM/" +\
                'OB3b_locus_tags_EC_number_no_KEGG_subsystems_filtered.pkl', 'rb')
temporal_file = pickle.load(pkl_file)
pkl_file.close()

print (len(temporal_file))

65


### Temporal_file 

In [580]:
for locus_tag in temporal_file:
    try:
        print(model_OB3b_old.genes.get_by_id(locus_tag).id, 
              model_OB3b_old.genes.get_by_id(locus_tag).reactions)
    except:
        pass      

CQW49_01205 frozenset({<Reaction CyLCyHORpp at 0x7f32911025c0>, <Reaction CYTBCYTC at 0x7f3290c26ef0>})
CQW49_01435 frozenset({<Reaction ANHMK at 0x7f329069dc18>})
CQW49_08960 frozenset({<Reaction NTRLASE at 0x7f32901cf0f0>})
CQW49_11545 frozenset({<Reaction KAS16 at 0x7f3290da3d30>, <Reaction KAS17 at 0x7f3290da3358>, <Reaction KAS13 at 0x7f3290da3668>, <Reaction KAS8 at 0x7f3290e06d68>, <Reaction KAS2 at 0x7f3290da3898>, <Reaction KAS7 at 0x7f3290e06eb8>})
CQW49_12190 frozenset({<Reaction METSOXR2 at 0x7f3290e94278>})
CQW49_13155 frozenset({<Reaction METSOXR1 at 0x7f3290e943c8>})
CQW49_15865 frozenset()
CQW49_17395 frozenset({<Reaction NI2uabcpp at 0x7f32901226a0>, <Reaction MG2uabcpp at 0x7f3290100fd0>})
CQW49_18215 frozenset({<Reaction NADPHQR2 at 0x7f3290ec3550>})
CQW49_21560 frozenset({<Reaction QRr at 0x7f3290fa2e10>})


CQW49_01205

In [587]:
for reaction in model_OB3b_old.genes.CQW49_01205.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

CyLCyHORpp Cytochrome c-555 (cL):cytochrome c-553(cH) oxidoreductase ficytcc553_p + focytcc555_p <=> ficytcc555_p + focytcc553_p
CYTBCYTC Cytochrome b Cytochrome c electron transfer ficytC_c + focytc_c <=> ficytb_c + focytC_c


In [586]:
gene_entry = genes["mtw:"+"CQW49_01205"]
print(gene_entry.orthology)

K00428  cytochrome c peroxidase [EC:1.11.1.5]


In [589]:
model_OB3b_old.reactions.CYTBCYTC

Reaction identifier,CYTBCYTC
Name,Cytochrome b Cytochrome c electron transfer
Memory address,0x07f3290c26ef0
Stoichiometry,ficytC_c + focytc_c <=> ficytb_c + focytC_c Ferricytochrome c + Ferrocytochrome c C42H53FeN8O6S2 <=> Ferricytochrome b + Ferrocytochrome C
GPR,CQW49_01205
Lower bound,-1000.0
Upper bound,1000.0


In [594]:
iMsOB3b.metabolites.focytc_c.summary()

PRODUCING REACTIONS -- reduced_cytochrome_c (focytc_c)
------------------------------------------------------
%       FLUX  RXN ID     REACTION
----  ------  ---------  --------------------------------------------------
90%    8.16   MEOHDH     2.0 ficytc_c + meoh_p --> fald_p + 2.0 focytc_c...
10%    0.944  UQCYOR_2p  2.0 ficytc_c + qh2_c --> 2.0 focytc_c + 2.0 h_p...

CONSUMING REACTIONS -- reduced_cytochrome_c (focytc_c)
------------------------------------------------------
%       FLUX  RXN ID     REACTION
----  ------  ---------  --------------------------------------------------
101%   9.18   CYO_2p     2.0 focytc_c + 4.0 h_c + 0.5 o2_c --> 2.0 ficyt...


CQW49_01435

In [597]:
for reaction in model_OB3b_old.genes.CQW49_01435.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

ANHMK 1,6-anhydrous-N-Acetylmuramate kinase anhm_c + atp_c + h2o_c --> acmum6p_c + adp_c + h_c


In [598]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.ANHMK)

In [599]:
iMsOB3b.reactions.ANHMK

Reaction identifier,ANHMK
Name,"1,6-anhydrous-N-Acetylmuramate kinase"
Memory address,0x07f329069dc18
Stoichiometry,"anhm_c + atp_c + h2o_c --> acmum6p_c + adp_c + h_c 1,6-anhydrous-N-Acetylmuramate + ATP + H2O --> N-acetylmuramate 6-phosphate + ADP + H_"
GPR,CQW49_01435
Lower bound,0.0
Upper bound,1000.0


CQW49_08960

In [600]:
for reaction in model_OB3b_old.genes.CQW49_08960.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

NTRLASE Nitrilase 2.0 h2o_c + ind3acnl_c --> ind3ac_c + nh4_c


In [602]:
model_OB3b_old.reactions.NTRLASE

Reaction identifier,NTRLASE
Name,Nitrilase
Memory address,0x07f32901cf0f0
Stoichiometry,2.0 h2o_c + ind3acnl_c --> ind3ac_c + nh4_c 2.0 H2O + indole-3-acetonitrile --> indole-3-acetate + Ammonium
GPR,CQW49_08960
Lower bound,0.0
Upper bound,1000.0


CQW49_11545 --> unclear EC number convention (nodulation protein E [EC:2.3.1.-])

In [603]:
for reaction in model_OB3b_old.genes.CQW49_11545.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

KAS16 3-hydroxy-myristoyl-ACP synthesis ddcaACP_c + 2.0 h_c + malACP_c + nadph_c --> 3hmrsACP_c + ACP_c + co2_c + nadp_c
KAS17 B-ketoacyl synthetase (n-C18:1) accoa_c + 22.0 h_c + 8.0 malcoa_c + 15.0 nadph_c --> 8.0 co2_c + 9.0 coa_c + 7.0 h2o_c + 15.0 nadp_c + ocdcea_c
KAS13 B-ketoacyl synthetase (octadecanoate) accoa_c + 23.0 h_c + 8.0 malcoa_c + 16.0 nadph_c --> 8.0 co2_c + 9.0 coa_c + 7.0 h2o_c + 16.0 nadp_c + ocdca_c
KAS8 B-ketoacyl synthetase (palmitate, n-C16:0) accoa_c + 20.0 h_c + 7.0 malcoa_c + 14.0 nadph_c --> 7.0 co2_c + 8.0 coa_c + 6.0 h2o_c + hdca_c + 14.0 nadp_c
KAS2 B-ketoacyl synthetase (n-C14:0) accoa_c + 17.0 h_c + 6.0 malcoa_c + 12.0 nadph_c --> 6.0 co2_c + 7.0 coa_c + 5.0 h2o_c + 12.0 nadp_c + ttdca_c
KAS7 B-ketoacyl synthetase (n-C16:1) accoa_c + 19.0 h_c + 7.0 malcoa_c + 13.0 nadph_c --> 7.0 co2_c + 8.0 coa_c + 6.0 h2o_c + hdcea_c + 13.0 nadp_c


CQW49_12190

In [605]:
for reaction in model_OB3b_old.genes.CQW49_12190.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

METSOXR2 L-methionine-R-sulfoxide reductase metsox_R__L_c + trdrd_c --> h2o_c + met__L_c + trdox_c


In [609]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.METSOXR2)

Ignoring reaction 'METSOXR2' since it already exists.


CQW49_13155

In [611]:
for reaction in model_OB3b_old.genes.CQW49_13155.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

METSOXR1 L-methionine-S-oxide reductase metsox_S__L_c + trdrd_c --> h2o_c + met__L_c + trdox_c


In [612]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.METSOXR2)

Ignoring reaction 'METSOXR2' since it already exists.


CQW49_15865 --> not added, no corresponding reaction

In [613]:
for reaction in model_OB3b_old.genes.CQW49_15865.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

CQW49_17395

In [615]:
for reaction in model_OB3b_old.genes.CQW49_17395.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

NI2uabcpp Nickel transport via ABC system (uptake, periplasm) atp_c + h2o_c + ni2_p --> adp_c + h_c + ni2_c + pi_c
MG2uabcpp Magnesium (Mg+2) ABC transporter (ubtake, periplasm) atp_c + h2o_c + mg2_p --> adp_c + h_c + mg2_c + pi_c


CQW49_18215

In [616]:
for reaction in model_OB3b_old.genes.CQW49_18215.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

NADPHQR2 NADPH Quinone Reductase (Ubiquinone-8) h_c + nadph_c + q8_c --> nadp_c + q8h2_c


In [619]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.NADPHQR2)

Ignoring reaction 'NADPHQR2' since it already exists.


In [628]:
iMsOB3b.reactions.get_by_id('NADPHQR2').reaction = 'h_c + nadph_c + q_c --> nadp_c + qh2_c'

CQW49_21560

In [634]:
for reaction in model_OB3b_old.genes.CQW49_21560.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

QRr QRr h_c + nadph_c + q_c --> nadp_c + qh2_c


In [638]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.QRr)

Ignoring reaction 'QRr' since it already exists.


Save model for now:

In [644]:
cobra.io.save_json_model(iMsOB3b, memote_directory + draft_gems_dir_8 + "bigg_OB3b_14_01_2020.json")

### OB3b_locus_tags_and_their_description_not_included_in_automatic_GEMs

In [637]:
for locus_tag in OB3b_locus_tags_and_their_description:
    try:
        print(model_OB3b_old.genes.get_by_id(locus_tag).id, 
              model_OB3b_old.genes.get_by_id(locus_tag).reactions)
    except:
        pass   

CQW49_00515 frozenset({<Reaction ATPSipp at 0x7f32910a10f0>})
CQW49_00550 frozenset({<Reaction ATPSipp at 0x7f32910a10f0>})
CQW49_00740 frozenset({<Reaction UACMAMO at 0x7f3290abfef0>})
CQW49_00915 frozenset({<Reaction BMOGDS2 at 0x7f3290c126a0>, <Reaction BMOGDS1 at 0x7f3290bfdcf8>})
CQW49_01170 frozenset({<Reaction DPR at 0x7f328fdc6588>})
CQW49_01220 frozenset({<Reaction PMMODCipp at 0x7f32910e5e80>, <Reaction PMMOipp at 0x7f32911026a0>})
CQW49_01360 frozenset({<Reaction NDPK4 at 0x7f3290142208>, <Reaction NDPK6 at 0x7f3290122e48>, <Reaction NDPK7 at 0x7f3290122c88>, <Reaction NDPK1 at 0x7f3290142eb8>, <Reaction NDPK5 at 0x7f32901428d0>, <Reaction NDPK2 at 0x7f3290142518>, <Reaction NDPK8 at 0x7f3290122b38>, <Reaction NDPK10 at 0x7f3290142748>, <Reaction NDPK3 at 0x7f3290142358>, <Reaction PYK at 0x7f3290277748>, <Reaction NDPK9 at 0x7f3290122978>})
CQW49_01705 frozenset({<Reaction LIPOS at 0x7f32908079e8>})
CQW49_01990 frozenset({<Reaction LPLIPAL1A160pp at 0x7f3290807048>, <Reacti

CQW49_00515

In [645]:
for reaction in model_OB3b_old.genes.CQW49_00515.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

ATPSipp ATP synthase F0 - H+ - complex V adp_c + 4.0 h_p + pi_c --> atp_c + h2o_c + 3.0 h_c


In [650]:
iMsOB3b.reactions.ATPS4rpp.gene_reaction_rule

'CQW49_19220 and CQW49_00300 and CQW49_19240 and CQW49_19235 and CQW49_00305 and CQW49_19230 and CQW49_19225 and CQW49_00310 and CQW49_00290 and CQW49_00295 and CQW49_00515 and CQW49_00550'

CQW49_00550

In [646]:
for reaction in model_OB3b_old.genes.CQW49_00550.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

ATPSipp ATP synthase F0 - H+ - complex V adp_c + 4.0 h_p + pi_c --> atp_c + h2o_c + 3.0 h_c


In [649]:
iMsOB3b.reactions.ATPS4rpp.gene_reaction_rule

'CQW49_19220 and CQW49_00300 and CQW49_19240 and CQW49_19235 and CQW49_00305 and CQW49_19230 and CQW49_19225 and CQW49_00310 and CQW49_00290 and CQW49_00295 and CQW49_00515 and CQW49_00550'

CQW49_00740

In [652]:
for reaction in model_OB3b_old.genes.CQW49_00740.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

UACMAMO UDP-N-acetyl-D-mannosamine oxidoreductase h2o_c + 2.0 nad_c + uacmam_c --> 3.0 h_c + 2.0 nadh_c + uacmamu_c


In [658]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.UACMAMO)

CQW49_00915

In [660]:
for reaction in model_OB3b_old.genes.CQW49_00915.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

BMOGDS2 Bis-molybdopterin guanine dinucleotide synthase bmoco1gdp_c + gtp_c + h_c --> bmocogdp_c + ppi_c
BMOGDS1 Bis-molybdopterin guanine dinucleotide synthase (single GDP) bmoco_c + gtp_c + h_c --> bmoco1gdp_c + ppi_c


CQW49_01170

In [662]:
for reaction in model_OB3b_old.genes.CQW49_01170.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

DPR 2-dehydropantoate 2-reductase 2dhp_c + h_c + nadph_c --> nadp_c + pant_R_c


In [665]:
iMsOB3b.reactions.DPR.gene_reaction_rule = 'CQW49_01170'

CQW49_01220

In [666]:
for reaction in model_OB3b_old.genes.CQW49_01220.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

PMMODCipp particulate Methane Monooxygenase - Direct Coupling ch4_p + 2.0 focytcc555_p + 2.0 h_p + o2_p --> 2.0 ficytcc555_p + h2o_p + meoh_p
PMMOipp particulate Methane Monooxygenase ch4_p + o2_p + q8h2_im --> h2o_p + meoh_p + q8_im


In [667]:
iMsOB3b.genes.CQW49_01220

Gene identifier,CQW49_01220
Name,
Memory address,0x07f329f2aff98
Functional,True
In 2 reaction(s),"PMMOipp, PMMODCipp"


CQW49_01360

In [668]:
for reaction in model_OB3b_old.genes.CQW49_01360.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

NDPK4 Nucleoside-diphosphate kinase (ATP:dTDP) atp_c + dtdp_c <=> adp_c + dttp_c
NDPK6 Nucleoside-diphosphate kinase (ATP:dUDP) atp_c + dudp_c <=> adp_c + dutp_c
NDPK7 Nucleoside-diphosphate kinase (ATP:dCDP) atp_c + dcdp_c <=> adp_c + dctp_c
NDPK1 Nucleoside-diphosphate kinase (ATP:GDP) adp_c + gtp_c <=> atp_c + gdp_c
NDPK5 Nucleoside-diphosphate kinase (ATP:dGDP) atp_c + dgdp_c <=> adp_c + dgtp_c
NDPK2 Nucleoside-diphosphate kinase (ATP:UDP) atp_c + udp_c --> adp_c + utp_c
NDPK8 Nucleoside-diphosphate kinase (ATP:dADP) atp_c + dadp_c --> adp_c + datp_c
NDPK10 Nucleoside-diphosphate kinase (ATP dIDP) atp_c + didp_c <=> adp_c + ditp_c
NDPK3 Nucleoside-diphosphate kinase (ATP:CDP) atp_c + cdp_c --> adp_c + ctp_c
PYK Pyruvate kinase adp_c + h_c + pep_c --> atp_c + pyr_c
NDPK9 Nucleoside diphosphate kinase  ATPIDP  atp_c + idp_c <=> adp_c + itp_c


In [673]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.NDPK1)
iMsOB3b.add_reaction(model_OB3b_old.reactions.NDPK2)
iMsOB3b.add_reaction(model_OB3b_old.reactions.NDPK3)
iMsOB3b.add_reaction(model_OB3b_old.reactions.NDPK4)
iMsOB3b.add_reaction(model_OB3b_old.reactions.NDPK5)
iMsOB3b.add_reaction(model_OB3b_old.reactions.NDPK6)
iMsOB3b.add_reaction(model_OB3b_old.reactions.NDPK7)
iMsOB3b.add_reaction(model_OB3b_old.reactions.NDPK8)
iMsOB3b.add_reaction(model_OB3b_old.reactions.NDPK9)
iMsOB3b.add_reaction(model_OB3b_old.reactions.NDPK10)

Ignoring reaction 'NDPK1' since it already exists.
Ignoring reaction 'NDPK2' since it already exists.
Ignoring reaction 'NDPK3' since it already exists.
Ignoring reaction 'NDPK4' since it already exists.
Ignoring reaction 'NDPK5' since it already exists.
Ignoring reaction 'NDPK6' since it already exists.
Ignoring reaction 'NDPK7' since it already exists.
Ignoring reaction 'NDPK8' since it already exists.
Ignoring reaction 'NDPK9' since it already exists.
Ignoring reaction 'NDPK10' since it already exists.


CQW49_01705

In [675]:
for reaction in model_OB3b_old.genes.CQW49_01705.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

LIPOS Lipoate synthase 4fe4s_c + 2.0 amet_c + h_c + nad_c + octapb_c --> 2fe2s_c + 2.0 dad_5_c + 2.0 fe2_c + lipopb_c + 2.0 met__L_c + nadh_c


In [677]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.LIPOS)

CQW49_01990

In [679]:
for reaction in model_OB3b_old.genes.CQW49_01990.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

LPLIPAL1A160pp Lysophospholipase L1 (2-acylglycerophosphotidate, n-C16:0) (periplasm) 1hdecg3p_p + h2o_p --> glyc3p_p + h_p + hdca_p
LPLIPAL1A140pp Lysophospholipase L1 (2-acylglycerophosphotidate, n-C14:0) (periplasm) 1tdecg3p_p + h2o_p --> glyc3p_p + h_p + ttdca_p
LPLIPAL1G160pp Lysophospholipase L1 (2-acylglycerophosphoglycerol, n-C16:0) (periplasm) 1agpg160_p + h2o_p --> g3pg_p + h_p + hdca_p
LPLIPAL1G140pp Lysophospholipase L1 (2-acylglycerophosphoglycerol, n-C14:0) (periplasm) 1agpg140_p + h2o_p --> g3pg_p + h_p + ttdca_p
LPLIPAL1E120pp Lysophospholipase L1 (2-acylglycerophosphoethanolamine, n-C12:0) (periplasm) 1agpe120_p + h2o_p --> ddca_p + g3pe_p + h_p
LPLIPAL1E141pp Lysophospholipase L1 (2-acylglycerophosphoethanolamine, n-C14:1) (periplasm) 1agpe141_p + h2o_p --> g3pe_p + h_p + ttdcea_p
LPLIPAL1E161pp Lysophospholipase L1 (2-acylglycerophosphoethanolamine, n-C16:1) (periplasm) 1agpe161_p + h2o_p --> g3pe_p + h_p + hdcea_p
LPLIPAL1A180pp Lysophospholipase L1 (2-acylglyceroph

In [688]:
for reaction in iMsOB3b.reactions:
    if "thioeste" in reaction.name:
        print(reaction.name, reaction.id, reaction.gene_reaction_rule)

B hydroxydecanoyl thioester dehydrase 3HAD100: 3hdecACP_c <=> h2o_c + tdec2eACP_c CQW49_00475 or CQW49_09310
acyl CoA thioesterase II  n h6 0  HFACOAE60: R_3hhcoa_c + h2o_c --> R_3hhc_c + coa_c + h_c CQW49_00725 or CQW49_04870
malyl CoA thioesterase MALCOATE: h2o_c + malScoa_c --> coa_c + h_c + mal_L_c CQW49_12840
acyl CoA thioesterase II  n 18 0  FACOAE180: h2o_c + stcoa_c --> coa_c + h_c + ocdca_c CQW49_00725 or CQW49_04870
acyl CoA thioesterase II  n 16 0  FACOAE160: h2o_c + pmtcoa_c --> coa_c + h_c + hdca_c CQW49_00725 or CQW49_04870
acyl CoA thioesterase II  n 14 0  FACOAE140: h2o_c + tdcoa_c --> coa_c + h_c + ttdca_c CQW49_00725 or CQW49_04870
acyl CoA thioesterase II  n h8 0  HFACOAE80: R_3hocoa_c + h2o_c --> R_3hoct_c + coa_c + h_c CQW49_00725 or CQW49_04870
acyl CoA thioesterase II  n 6 0  FACOAE60: h2o_c + hxcoa_c --> coa_c + h_c + hxa_c CQW49_00725 or CQW49_04870
acyl CoA thioesterase II  n 8 0  FACOAE80: h2o_c + occoa_c --> coa_c + h_c + octa_c CQW49_00725 or CQW49_04870
ac

In [689]:
iMsOB3b.reactions.HFACOAE60.gene_reaction_rule = 'CQW49_00725 or CQW49_04870 or CQW49_01990'
iMsOB3b.reactions.FACOAE180.gene_reaction_rule = 'CQW49_00725 or CQW49_04870 or CQW49_01990'
iMsOB3b.reactions.FACOAE160.gene_reaction_rule = 'CQW49_00725 or CQW49_04870 or CQW49_01990'
iMsOB3b.reactions.FACOAE140.gene_reaction_rule = 'CQW49_00725 or CQW49_04870 or CQW49_01990'
iMsOB3b.reactions.HFACOAE80.gene_reaction_rule = 'CQW49_00725 or CQW49_04870 or CQW49_01990'
iMsOB3b.reactions.FACOAE60.gene_reaction_rule = 'CQW49_00725 or CQW49_04870 or CQW49_01990'
iMsOB3b.reactions.FACOAE80.gene_reaction_rule = 'CQW49_00725 or CQW49_04870 or CQW49_01990'
iMsOB3b.reactions.HFACOAE180.gene_reaction_rule = 'CQW49_00725 or CQW49_04870 or CQW49_01990'
iMsOB3b.reactions.HFACOAE120.gene_reaction_rule = 'CQW49_00725 or CQW49_04870 or CQW49_01990'
iMsOB3b.reactions.FACOAE120.gene_reaction_rule = 'CQW49_00725 or CQW49_04870 or CQW49_01990'
iMsOB3b.reactions.HFACOAE100.gene_reaction_rule = 'CQW49_00725 or CQW49_04870 or CQW49_01990'
iMsOB3b.reactions.FACOAE100.gene_reaction_rule = 'CQW49_00725 or CQW49_04870 or CQW49_01990'
iMsOB3b.reactions.HFACOAE160.gene_reaction_rule = 'CQW49_00725 or CQW49_04870 or CQW49_01990'
iMsOB3b.reactions.HFACOAE140.gene_reaction_rule = 'CQW49_00725 or CQW49_04870 or CQW49_01990'

CQW49_02400

In [690]:
for reaction in model_OB3b_old.genes.CQW49_02400.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

GTHRDHpp Glutathione hydralase (periplasmic) gthrd_c + h2o_c --> cgly_c + glu__L_c


In [692]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.GTHRDHpp)

CQW49_02415

In [693]:
for reaction in model_OB3b_old.genes.CQW49_02415.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

PPS Phosphoenolpyruvate synthase atp_c + h2o_c + pyr_c --> amp_c + 2.0 h_c + pep_c + pi_c


In [694]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.PPS)

CQW49_02635

In [697]:
for reaction in model_OB3b_old.genes.CQW49_02635.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

ORNTA Ornithine transaminase akg_c + orn_c --> glu5sa_c + glu__L_c
PTRCTA Putrescine Transaminase akg_c + ptrc_c --> 4abutn_c + glu__L_c


In [703]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.ORNTA)

CQW49_02865
<br> build reaction based on http://bigg.ucsd.edu/universal/reactions/CLS_m
<br> these reactions belong to glycerolphospholipid metabolism

In [704]:
for reaction in model_OB3b_old.genes.CQW49_02865.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

PGSA_MCi Phosphatidylglycerol synthase 0.001 cdpdag_MC_c + glyc1p_c --> cmp_c + h_c + 0.001 pgp_MC_c


In [713]:
import re
for metabolite in iMsOB3b.metabolites:
    if re.search("cardiolipin", metabolite.name, re.IGNORECASE):
        print(metabolite.id, metabolite.name)

clpnd181_c cardiolipin_dioctadec_9_enoyl_
clpnd180_c cardiolipin_dioctadecanoyl_
clpnd160_c cardiolipin_dihexadecanoyl_
clpnd161_c cardiolipin_dihexadec_9_enoyl_


In [714]:
iMsOB3b.metabolites.clpnd161_c

Metabolite identifier,clpnd161_c
Name,cardiolipin_dihexadec_9_enoyl_
Memory address,0x07f329bdaae80
Formula,None
Compartment,c
In 0 reaction(s),


In [716]:
import re
for metabolite in iMsOB3b.metabolites:
    if re.search("phosphatidylglycerol", metabolite.name, re.IGNORECASE):
        print(metabolite.id, metabolite.name)

pg161_c phosphatidylglycerol_dihexadec_9_enoyl_
pgp160_c phosphatidylglycerolphosphate_dihexadecanoyl_
pg180_c phosphatidylglycerol_dioctadecanoyl_
pgp181_c phosphatidylglycerolphosphate_dioctadec_9_enoyl_
pg181_c phosphatidylglycerol_dioctadec_9_enoyl_
pg160_c phosphatidylglycerol_dihexadecanoyl_
pgp161_c phosphatidylglycerolphosphate_dihexadec_9_enoyl_
pgp180_c phosphatidylglycerolphosphate_dioctadecanoyl_


In [717]:
iMsOB3b.metabolites.pg160_c

Metabolite identifier,pg160_c
Name,phosphatidylglycerol_dihexadecanoyl_
Memory address,0x07f329eba8710
Formula,None
Compartment,c
In 2 reaction(s),"Biomass_Mextorquens_AM1_core, PGPP160"


In [720]:
iMsOB3b.reactions.PGPP160

Reaction identifier,PGPP160
Name,phosphatidylglycerophosphatase n 16 0
Memory address,0x07f329b4038d0
Stoichiometry,h2o_c + pgp160_c <=> pg160_c + pi_c H2O + phosphatidylglycerolphosphate_dihexadecanoyl_ <=> phosphatidylglycerol_dihexadecanoyl_ + phosphate
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [724]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("CDP", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

RNDR3 CDP reductase
CDPGD cdp glucose 4 6 dehydratase
NDPK7 dCDP kinase
NDPK3 CDP kinase
DASYN160 CDP diacylglycerol synthetase  n 16 0 
DASYN161 CDP diacylglycerol synthetase  n 16 1 
DASYN180 CDP diacylglycerol synthetase  n 18 0 
DASYN181 CDP diacylglycerol synthetase  n 18 1 


In [732]:
#iMsOB3b.reactions.DASYN160
#iMsOB3b.reactions.DASYN161
#iMsOB3b.reactions.DASYN180
iMsOB3b.reactions.DASYN181

Reaction identifier,DASYN181
Name,CDP diacylglycerol synthetase n 18 1
Memory address,0x07f329f96ee10
Stoichiometry,ctp_c + h_c + pa181_c --> cdpdodec11eg_c + ppi_c CTP + H_ + 1_2_dioctadec_9_enoyl_sn_glycerol_3_phosphate --> CDP_1_2_dioctadec_9_enoyl_glycerol + diphosphate
GPR,CQW49_00450
Lower bound,0.0
Upper bound,1000.0


In [ ]:
iMsOB3b.reactions.CLPNS160

iMsOB3b.reactions.CLPNS161

iMsOB3b.reactions.CLPNS180

iMsOB3b.reactions.CLPNS181

cdp12dgr160_m + pg160_m ⇌ cmp_m + h_m + clpn160_m

In [735]:
from cobra import Model, Reaction, Metabolite
# Best practise: SBML compliant IDs

reaction = Reaction('CLPNS160')
reaction.name = 'Cardiolipin synthase (n-16:0)'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default

iMsOB3b.add_reaction(reaction)
iMsOB3b.reactions.CLPNS160.reaction = 'cdpdhdecg_c + pg160_c <=> cmp_c + clpnd160_c'

Ignoring reaction 'CLPNS160' since it already exists.


In [736]:
from cobra import Model, Reaction, Metabolite
# Best practise: SBML compliant IDs

reaction = Reaction('CLPNS161')
reaction.name = 'Cardiolipin synthase (n-16:1)'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default

iMsOB3b.add_reaction(reaction)
iMsOB3b.reactions.CLPNS161.reaction = 'cdpdhdec9eg_c + pg161_c <=> cmp_c + clpnd161_c'

In [737]:
from cobra import Model, Reaction, Metabolite
# Best practise: SBML compliant IDs

reaction = Reaction('CLPNS180')
reaction.name = 'Cardiolipin synthase (n-18:0)'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default

iMsOB3b.add_reaction(reaction)
iMsOB3b.reactions.CLPNS180.reaction = 'cdpdodecg_c + pg180_c <=> cmp_c + clpnd180_c'

In [738]:
from cobra import Model, Reaction, Metabolite
# Best practise: SBML compliant IDs

reaction = Reaction('CLPNS181')
reaction.name = 'Cardiolipin synthase (n-18:1)'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default

iMsOB3b.add_reaction(reaction)
iMsOB3b.reactions.CLPNS181.reaction = 'cdpdodec11eg_c + pg181_c <=> cmp_c + clpnd181_c'

In [740]:
iMsOB3b.reactions.CLPNS160.gene_reaction_rule = 'CQW49_02865'
iMsOB3b.reactions.CLPNS161.gene_reaction_rule = 'CQW49_02865'
iMsOB3b.reactions.CLPNS180.gene_reaction_rule = 'CQW49_02865'
iMsOB3b.reactions.CLPNS181.gene_reaction_rule = 'CQW49_02865'

CQW49_02960

In [741]:
for reaction in model_OB3b_old.genes.CQW49_02960.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

SHSL1 O-succinylhomoserine lyase (L-cysteine) cys__L_c + suchms_c --> cyst__L_c + h_c + succ_c
SHSL2 o-succinylhomoserine sulfhydrylase h2s_c + suchms_c --> h_c + lhcys_c + succ_c
SHSL2r SHSL2r h2s_c + suchms_c --> h_c + hcys__L_c + succ_c


In [742]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.SHSL2)

CQW49_03300

In [743]:
for reaction in model_OB3b_old.genes.CQW49_03300.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

CA2abcpp Calcium transport via ABC system (periplasm) atp_c + ca2_p + h2o_c --> adp_c + ca2_c + h_c + pi_c


In [745]:
iMsOB3b.reactions.CA2abctpp.gene_reaction_rule = 'CQW49_03300'

CQW49_03485

In [746]:
for reaction in model_OB3b_old.genes.CQW49_03485.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

NADH16pp type-1 nadh:quinone oxidoreductase (H+-translocating) 4.0 h_c + nadh_c + q8_im --> 3.0 h_p + nad_c + q8h2_im


In [749]:
iMsOB3b.reactions.NADH11.gene_reaction_rule = 'CQW49_05325 and CQW49_05330 and CQW49_18615 and CQW49_05310 and CQW49_05315 and CQW49_05300 and CQW49_05305 and CQW49_05290 and CQW49_05335 and CQW49_05295 and CQW49_05340 and CQW49_05345 and CQW49_05285 and CQW49_05350 and CQW49_05320 and CQW49_03485'

CQW49_03930

In [750]:
for reaction in model_OB3b_old.genes.CQW49_03930.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

GUAPRT Guanine phosphoribosyltransferase gua_c + prpp_c --> gmp_c + ppi_c
HXPRT Hypoxanthine phosphoribosyltransferase (Hypoxanthine) hxan_c + prpp_c --> imp_c + ppi_c
XPPT Xanthine phosphoribosyltransferase prpp_c + xan_c --> ppi_c + xmp_c


In [753]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.XPPT)

CQW49_04245
<br> Add reaction since it is present in genbank annotation

In [755]:
for reaction in model_OB3b_old.genes.CQW49_04245.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

MSAR Malonic semialdehyde reductase h_c + msa_c + nadph_c --> 3hpp_c + nadp_c


In [758]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.MSAR)

CQW49_04395

In [759]:
for reaction in model_OB3b_old.genes.CQW49_04395.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

CQW49_04465

In [760]:
for reaction in model_OB3b_old.genes.CQW49_04465.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

CQW49_05060 --> too broad definition

In [762]:
for reaction in model_OB3b_old.genes.CQW49_05060.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

AMPTASECG Alanyl aminopeptidase (cys-gly) cgly_c + h2o_c --> cys__L_c + gly_c
AMPTASEPG Aminopeptidase (pro-gly) h2o_c + progly_c --> gly_c + pro__L_c


CQW49_05095

In [773]:
for reaction in model_OB3b_old.genes.CQW49_05095.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

PPC Phosphoenolpyruvate carboxylase co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c


In [776]:
iMsOB3b.reactions.PPC.gene_reaction_rule

'CQW49_05095 or CQW49_12230'

CQW49_05280

In [777]:
for reaction in model_OB3b_old.genes.CQW49_05280.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

BACCL Biotin acetyl CoA carboxylase ligase atp_c + btn_c + h_c --> btamp_c + ppi_c


In [778]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.BACCL)

CQW49_05640

In [784]:
for reaction in model_OB3b_old.genes.CQW49_05640.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

HGNTOR Homogentisateoxygen 1 2 oxidoreductase  decyclizing  hgentis_c + o2_c --> 4mlacac_c + h_c


In [783]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.HGNTOR)

Ignoring reaction 'HGNTOR' since it already exists.


CQW49_05645

In [786]:
for reaction in model_OB3b_old.genes.CQW49_05645.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

34HPPOR 4 Hydroxyphenylpyruvateoxygen oxidoreductase 34hpp_c + o2_c --> co2_c + hgentis_c


In [789]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.get_by_id('34HPPOR'))

Ignoring reaction '34HPPOR' since it already exists.


CQW49_06015

In [790]:
for reaction in model_OB3b_old.genes.CQW49_06015.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

THIORDXi Hydrogen peroxide reductase (thioredoxin) h2o2_c + trdrd_c --> 2.0 h2o_c + trdox_c


In [794]:
import re
for metabolite in iMsOB3b.metabolites:
    if re.search("glutathione", metabolite.name, re.IGNORECASE):
        print(metabolite.id, metabolite.name)

lgt_S_c S_lactoylglutathione
gthrd_c glutathione
gthox_c glutathione_disulfide


In [799]:
iMsOB3b.reactions.GTHPp.gene_reaction_rule = "CQW49_06010 or CQW49_06015"

Save model for now:

In [800]:
cobra.io.save_json_model(iMsOB3b, memote_directory + draft_gems_dir_8 + "bigg_OB3b_15_01_2020.json")

CQW49_06060 
<br> too broad definition
<br> not added

In [804]:
for reaction in model_OB3b_old.genes.CQW49_06060.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

AMPTASEPG Aminopeptidase (pro-gly) h2o_c + progly_c --> gly_c + pro__L_c


CQW49_06175

In [806]:
for reaction in model_OB3b_old.genes.CQW49_06175.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

FTHFCL 5 formethyltetrahydrofolate cyclo ligase 5fthf_c + atp_c --> adp_c + methf_c + pi_c


In [809]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.FTHFCL)

CQW49_06290

In [810]:
for reaction in model_OB3b_old.genes.CQW49_06290.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

CPPPGO2 Oxygen Independent coproporphyrinogen-III oxidase 2.0 amet_c + cpppg3_c --> 2.0 co2_c + 2.0 dad_5_c + 2.0 met__L_c + pppg9_c


In [818]:
print(iMsOB3b.metabolites.cpppg3_c.name)
print(iMsOB3b.metabolites.amet_c.name)
print(iMsOB3b.metabolites.dad_5_c.name)
print(iMsOB3b.metabolites.pppg9_c.name)

coproporphyrinogen_III
S__adenosyl_L_methionine
5'-deoxyadenosine
Protoporphyrinogen_IX


In [819]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.CPPPGO2)

CQW49_06695

In [820]:
for reaction in model_OB3b_old.genes.CQW49_06695.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

ORNDC Ornithine Decarboxylase h_c + orn_c --> co2_c + ptrc_c


In [821]:
import re
for metabolite in iMsOB3b.metabolites:
    if re.search("ornithine", metabolite.name, re.IGNORECASE):
        print(metabolite.id, metabolite.name)

orn_c L_ornithine
sucorn_c N2_succinyl_L_ornithine
acorn_c N_acetyl_L_ornithine


In [823]:
iMsOB3b.metabolites.orn_c.reactions

frozenset({<Reaction ACODA at 0x7f329f700588>,
           <Reaction GLYAMDTRc at 0x7f329b4700b8>,
           <Reaction OCBT at 0x7f329f5e08d0>,
           <Reaction ORNDC at 0x7f329f4fdc88>,
           <Reaction ORNTA at 0x7f32901aee80>,
           <Reaction ORNTAC at 0x7f329b183dd8>})

In [827]:
iMsOB3b.reactions.ORNDC.gene_reaction_rule = 'CQW49_06695'

CQW49_06715
<br> change reaction bounds so that nh4_c is not assimilated via glutamate dehydrogenase

In [828]:
for reaction in model_OB3b_old.genes.CQW49_06715.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

GLUDy Glutamate dehydrogenase (NADP) glu__L_c + h2o_c + nadp_c <=> akg_c + h_c + nadph_c + nh4_c
GLUDxi Glutamate dehydrogenase  NAD  glu__L_c + h2o_c + nad_c --> akg_c + h_c + nadh_c + nh4_c


In [831]:
import re
for metabolite in iMsOB3b.metabolites:
    if re.search("glutamate", metabolite.name, re.IGNORECASE):
        print(metabolite.id, metabolite.name)

glu_D_c D_glutamate
glu_L_e L_glutamate
glu_L_p L_glutamate
glu_L_c L_glutamate
uamag_c UDP_N_acetylmuramoyl_L_alanyl_D_glutamate
Nforglu_c N_formyl_L_glutamate
glu5p_c L_glutamate_5_phosphate
acg5sa_c N_acetyl_L_glutamate_5_semialdehyde
sucgsa_c N2_succinyl_L_glutamate_5_semialdehyde
acglu_c N_acetyl_L_glutamate
glu5sa_c L_glutamate_gamma_semialdehyde
glu__L_c L-glutamate


In [835]:
for reaction in iMsOB3b.metabolites.glu_D_c.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

UAMAGS UDP N acetylmuramoylalanine  D glutamate ligase atp_c + glu_D_c + uama_c --> adp_c + h_c + pi_c + uamag_c
GLUR glutamate racemase glu_L_c <=> glu_D_c
ALATA_D D alanine aminotransferase glu_D_c + pyr_c <=> akg_c + ala_D_c


In [840]:
print(iMsOB3b.reactions.GLNS)
print(iMsOB3b.reactions.GLUSy)

GLNS: atp_c + glu_L_c + nh4_c --> adp_c + gln_L_c + h_c + pi_c
GLUSy: akg_c + gln_L_c + h_c + nadph_c --> 2.0 glu_L_c + nadp_c


In [841]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.GLUDy)
iMsOB3b.add_reaction(model_OB3b_old.reactions.GLUDxi)

In [846]:
iMsOB3b.reactions.GLUDy.bounds = (0, 1000)
iMsOB3b.reactions.GLUDxi.bounds = (0, 1000)

CQW49_06730

In [847]:
for reaction in model_OB3b_old.genes.CQW49_06730.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

OIVD2 2-oxoisovalerate dehydrogenase (acylating; 3-methyl-2-oxobutanoate) 3mob_c + coa_c + nad_c --> co2_c + ibcoa_c + nadh_c
OBDHc OBDHc 2obut_c + coa_c + nad_c --> co2_c + nadh_c + ppcoa_c
OIVD3 2-oxoisovalerate dehydrogenase (acylating; 3-methyl-2-oxopentanoate) 3mop_c + coa_c + nad_c --> 2mbcoa_c + co2_c + nadh_c


In [848]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("pyruvate", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

EX_pyr_e exchange flux   pyruvate
PYRtex pyruvate transport via diffusion  extracellular to periplasm 
PPDK pyruvate phosphate dikinase
PYK pyruvate kinase
MCPST 3 mercaptopyruvate sulfurtransferase
PPC phosphoenolpyruvate carboxylase
PYRt2rpp pyruvate symport  periplasm to cytoplasm 
PDH pyruvate dehydrogenase
GLYD hydroxypyruvate reductase
PPS Phosphoenolpyruvate synthase
34HPPOR 4 Hydroxyphenylpyruvateoxygen oxidoreductase


In [851]:
iMsOB3b.reactions.PDH.gene_reaction_rule = 'CQW49_01940 and CQW49_01945 and CQW49_01950 and CQW49_06860 and CQW49_06730'

CQW49_06735
<br> "Dihydrolipoyl transacetylase (or dihydrolipoamide acetyltransferase) is an enzyme component of the multienzyme pyruvate dehydrogenase complex. The pyruvate dehydrogenase complex is responsible for the pyruvate decarboxylation step that links glycolysis to the citric acid cycle. This involves the transformation of pyruvate from glycolysis into acetyl-CoA which is then used in the citric acid cycle to carry out cellular respiration." --> https://en.wikipedia.org/wiki/Dihydrolipoyl_transacetylase

In [852]:
for reaction in model_OB3b_old.genes.CQW49_06735.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

OIVD2 2-oxoisovalerate dehydrogenase (acylating; 3-methyl-2-oxobutanoate) 3mob_c + coa_c + nad_c --> co2_c + ibcoa_c + nadh_c
OBDHc OBDHc 2obut_c + coa_c + nad_c --> co2_c + nadh_c + ppcoa_c
OIVD3 2-oxoisovalerate dehydrogenase (acylating; 3-methyl-2-oxopentanoate) 3mop_c + coa_c + nad_c --> 2mbcoa_c + co2_c + nadh_c


In [857]:
iMsOB3b.reactions.PDH.gene_reaction_rule = 'CQW49_01940 and CQW49_01945 and CQW49_01950 and CQW49_06860 and CQW49_06730 and CQW49_06735'

CQW49_06750

In [858]:
for reaction in model_OB3b_old.genes.CQW49_06750.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

HPYRRy Hydroxypyruvate reductase (NADPH) h_c + hpyr_c + nadph_c --> glyc__R_c + nadp_c
2DGULRx 2-dehydro-L-gulonate reductase (NADH) 2dhguln_c + h_c + nadh_c --> idon__L_c + nad_c
2DGULRGy 2-Dehydro-L-gulonate reductase to gluconate (NADPH) 2dhguln_c + h_c + nadph_c --> glcn_c + nadp_c
DKGLCNR2x 2,5-diketo-D-gluconate reductase (NADH) 25dkglcn_c + h_c + nadh_c --> 5dglcn_c + nad_c
2DGULRy 2-dehydro-L-gulonate reductase (NADPH) 2dhguln_c + h_c + nadph_c --> idon__L_c + nadp_c
HPYRRx Hydroxypyruvate reductase (NADH) h_c + hpyr_c + nadh_c --> glyc__R_c + nad_c
2DGULRGx 2-Dehydro-L-gulonate reductase to gluconate (NADH) 2dhguln_c + h_c + nadh_c --> glcn_c + nad_c
DKGLCNR2y 2,5-diketo-D-gluconate reductase (NADPH) 25dkglcn_c + h_c + nadph_c --> 5dglcn_c + nadp_c


In [859]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("glyoxylate", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

GLYCLTDx glyoxylate reductase
GLXDH glyoxylate dehydrogenase
SERGLXAT L serine glyoxylate aminotransferase


In [862]:
iMsOB3b.reactions.GLYCLTDx.gene_reaction_rule = 'CQW49_04355 and CQW49_04350 and CQW49_04360 and CQW49_06750'

CQW49_06900

In [863]:
for reaction in model_OB3b_old.genes.CQW49_06900.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

UDPGDC UDP-glucuronate C-4'' decarboxylase nad_c + udpglcur_c --> co2_c + nadh_c + udpLa4o_c
TDPGDH DTDPglucose 4,6-dehydratase dtdpglu_c --> dtdp4d6dg_c + h2o_c
ULA4NFT UDP-L-Ara4N formyltransferase 10fthf_c + udpLa4n_c --> h_c + thf_c + udpLa4fn_c


In [866]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.UDPGDC)

CQW49_07145

In [867]:
for reaction in model_OB3b_old.genes.CQW49_07145.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

HSTPT Histidinol-phosphate transaminase glu__L_c + imacp_c --> akg_c + hisp_c
HSTPTr histidinol phosphate transaminase imactp_c + lglu_c --> akg_c + hispP_c


In [868]:
iMsOB3b.metabolites.hisp_c

Metabolite identifier,hisp_c
Name,L_histidinol_phosphate
Memory address,0x07f329bcc2278
Formula,None
Compartment,c
In 2 reaction(s),"HISTP, HSTPTr"


In [869]:
iMsOB3b.metabolites.imacp_c

Metabolite identifier,imacp_c
Name,imidazol_acetol_phosphate
Memory address,0x07f329bcff240
Formula,None
Compartment,c
In 2 reaction(s),"IGPDH, HSTPTr"


In [871]:
iMsOB3b.reactions.HSTPTr.gene_reaction_rule = "CQW49_07145 or CQW49_08850"

CQW49_07785
<br> 'DNA (cytosine-5)-methyltransferase 3A is an enzyme that catalyzes the transfer of methyl groups to specific CpG structures in DNA, a process called DNA methylation.' --> https://en.wikipedia.org/wiki/DNA_(cytosine-5)-methyltransferase_3A

In [872]:
for reaction in model_OB3b_old.genes.CQW49_07785.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

CYTOM Cytosine 5 methyltransferase amet_c + csn_c <=> 5mcsn_c + ahcys_c + h_c


CQW49_08660

In [874]:
for reaction in model_OB3b_old.genes.CQW49_08660.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

SERAT serine acetyltransferase accoa_c + lser_c --> acser_c + coa_c


In [881]:
iMsOB3b.reactions.SERAT.gene_reaction_rule = 'CQW49_00250 or CQW49_08660 or (CQW49_00250 and CQW49_10340)'

CQW49_08665

In [882]:
for reaction in model_OB3b_old.genes.CQW49_08665.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

HCITS Homocitrate synthase accoa_c + akg_c + h2o_c --> coa_c + h_c + hcit_c


In [884]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.HCITS)

CQW49_08720 and CQW49_08725

In [885]:
for reaction in model_OB3b_old.genes.CQW49_08720.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

NIT1b Nitrogenase 16.0 atp_c + 4.0 fdxr_42_c + 16.0 h2o_c + n2_c --> 16.0 adp_c + 4.0 fdxo_42_c + h2_c + 6.0 h_c + 2.0 nh4_c + 16.0 pi_c
NIT_mc Nitrogenase 16.0 atp_c + 4.0 fdxrd_c + 16.0 h2o_c + n2_c --> 16.0 adp_c + 4.0 fdxox_c + h2_c + 14.0 h_c + 2.0 nh4_c + 16.0 pi_c


In [889]:
for reaction in model_OB3b_old.genes.CQW49_08725.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

NIT1b Nitrogenase 16.0 atp_c + 4.0 fdxr_42_c + 16.0 h2o_c + n2_c --> 16.0 adp_c + 4.0 fdxo_42_c + h2_c + 6.0 h_c + 2.0 nh4_c + 16.0 pi_c
NIT_mc Nitrogenase 16.0 atp_c + 4.0 fdxrd_c + 16.0 h2o_c + n2_c --> 16.0 adp_c + 4.0 fdxox_c + h2_c + 14.0 h_c + 2.0 nh4_c + 16.0 pi_c


In [887]:
iMsOB3b.reactions.NIT_mc.gene_reaction_rule = 'CQW49_08720 and CQW49_08725'

CQW49_09160

In [890]:
for reaction in model_OB3b_old.genes.CQW49_09160.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

CYSDS Cysteine Desulfhydrase cys__L_c + h2o_c --> h2s_c + nh4_c + pyr_c
TRPAS2 Tryptophanase (L-tryptophan) h2o_c + trp__L_c <=> indole_c + nh4_c + pyr_c
SERD_L L-serine deaminase ser__L_c --> nh4_c + pyr_c


In [891]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.TRPAS2)

In [906]:
iMsOB3b.reactions.TRPAS2.bounds = (0, 1000)
iMsOB3b.reactions.TRPAS2.reaction = 'h2o_c + trp_L_c --> indole_c + nh4_c + pyr_c'

CQW49_09295
<br> https://www.genome.jp/dbget-bin/www_bget?mtw:CQW49_09295


CQW49_09405
<br> (GenBank) complex I NDUFA9 subunit family protein --> not added
https://www.genome.jp/dbget-bin/www_bget?mtw:CQW49_09405

In [910]:
for reaction in model_OB3b_old.genes.CQW49_09405.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

NADH16pp type-1 nadh:quinone oxidoreductase (H+-translocating) 4.0 h_c + nadh_c + q8_im --> 3.0 h_p + nad_c + q8h2_im


CQW49_09415

In [912]:
for reaction in model_OB3b_old.genes.CQW49_09415.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

NTPTP1 Nucleoside triphosphate tripolyhydrolase dgtp_c + h2o_c --> dgsn_c + pppi_c
NTPTP2 Nucleoside triphosphate tripolyhydrolase gtp_c + h2o_c --> gsn_c + pppi_c


In [913]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.NTPTP1)

CQW49_09715

In [914]:
for reaction in model_OB3b_old.genes.CQW49_09715.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

HOPNTAL 4-hydroxy-2-oxopentanoate aldolase 4h2opntn_c --> acald_c + pyr_c


In [917]:
import re
for metabolite in iMsOB3b.metabolites:
    if re.search("oxopentanoate", metabolite.name, re.IGNORECASE):
        print(metabolite.id, metabolite.name)

4mop_c 4_methyl_2_oxopentanoate


In [918]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.HOPNTAL)

CQW49_09940

In [919]:
for reaction in model_OB3b_old.genes.CQW49_09940.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

PGI Glucose-6-phosphate isomerase f6p_c <=> g6p_c
PGIA Glucose-6-phosphate isomerase (g6p-A) f6p_c <=> g6p_A_c
G6PI3 Glucose 6 phosphate isomerase g6p_B_c <=> f6p_c
G6PI Glucose 6 phosphate isomerase g6p_c <=> g6p_B_c
G6PI_1 Glucose-6-phosphate isomerase g6p_A_c <=> g6p_c


In [923]:
import re
for metabolite in iMsOB3b.metabolites:
    if re.search("glucose", metabolite.name, re.IGNORECASE):
        print(metabolite.id, metabolite.name)

glc_D_p D_glucose
cdpddglc_c CDP_4_dehydro_6_deoxy_D_glucose
glc_D_c D_glucose
glc_D_e D_glucose
g1p_D_B_c b_D_glucose_1_phosphate
dtdpglu_c dTDP_glucose
g6p_B_c b_D_glucose_6_phosphate
g6p_c D_glucose_6_phosphate
cdpd2dglc_c CDP_4_dehydro_3_6_dideoxy_D_glucose
udpg_c UDP_glucose
adpglc_c ADP_glucose
dtdp4d6dg_c dTDP_4_dehydro_6_deoxy_D_glucose
g1p_c D_glucose_1_phosphate
cdpglc_c CDP_glucose
b_glc_D_c b_D_glucose


In [926]:
for reaction in iMsOB3b.metabolites.f6p_c.reactions:
    print(reaction)

FBP: fdp_c + h2o_c --> f6p_c + pi_c
MAN6PI: f6p_c <=> man6p_c
F6PPKL: f6p_c + pi_c <=> actp_c + e4p_c + h2o_c
PGI: f6p_c <=> g6p_c
GF6PTAr: f6p_c + gln_L_c --> gam6p_c + glu_L_c
TALA: g3p_c + s7p_c <=> e4p_c + f6p_c


In [929]:
iMsOB3b.reactions.PGI.gene_reaction_rule = 'CQW49_09940'

CQW49_10130

In [930]:
for reaction in model_OB3b_old.genes.CQW49_10130.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

CCGS 7-cyano-7-carbaguanine synthase atp_c + cdg_c + nh4_c --> adp_c + h2o_c + h_c + pi_c + preq0_c


In [931]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.CCGS)

In [933]:
iMsOB3b.reactions.CCGS.bounds = (-1000, 0)

CQW49_10255

In [935]:
for reaction in model_OB3b_old.genes.CQW49_10255.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

PMMODCipp particulate Methane Monooxygenase - Direct Coupling ch4_p + 2.0 focytcc555_p + 2.0 h_p + o2_p --> 2.0 ficytcc555_p + h2o_p + meoh_p
PMMOipp particulate Methane Monooxygenase ch4_p + o2_p + q8h2_im --> h2o_p + meoh_p + q8_im
pMMO particulate methane monooxygenase ch4_p + elacrd_p + o2_p --> elaco_p + h2o_p + meoh_p


In [936]:
iMsOB3b.genes.CQW49_10255

Gene identifier,CQW49_10255
Name,
Memory address,0x07f329f2afa90
Functional,True
In 2 reaction(s),"PMMOipp, PMMODCipp"


CQW49_10875

In [937]:
for reaction in model_OB3b_old.genes.CQW49_10875.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

HCO3E HCO3 equilibration reaction co2_c + h2o_c <=> h_c + hco3_c


In [940]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("carbonic", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

HCO3E carbonic anhydrase


In [942]:
iMsOB3b.reactions.HCO3E.gene_reaction_rule = 'CQW49_10875 or CQW49_20715'

CQW49_10980

In [943]:
for reaction in model_OB3b_old.genes.CQW49_10980.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

DAAD D-Amino acid dehydrogenase ala__D_c + fad_c + h2o_c --> fadh2_c + nh4_c + pyr_c


In [944]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.DAAD)

CQW49_12035

In [945]:
for reaction in model_OB3b_old.genes.CQW49_12035.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

NAPRT NAPRTase h_c + nac_c + prpp_c --> nicrnt_c + ppi_c


In [946]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("nicotinate", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

NNDMBRT nicotinate nucleotide  dimethylbenzimidazole phosphoribosyltransferase
NNATr nicotinate nucleotide adenylyltransferase
NAPRTr nicotinate phosphoribosyltransferase


In [948]:
iMsOB3b.reactions.NAPRTr.gene_reaction_rule = 'CQW49_12035'

CQW49_12245

In [949]:
for reaction in model_OB3b_old.genes.CQW49_12245.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

GLYCK2 Glycerate kinase atp_c + glyc__R_c --> 2pg_c + adp_c + h_c


In [951]:
iMsOB3b.reactions.GLYCK2.gene_reaction_rule

'CQW49_12245'

CQW49_12295
inconsistent annotation for this gene --> https://www.genome.jp/dbget-bin/www_bget?ec:3.7.1.20

In [952]:
for reaction in model_OB3b_old.genes.CQW49_12295.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

OPTCCL 5 carboxy 2 oxohept 3 enedioate decarboxylation 5cohe_c + h_c --> 2hh24dd_c + co2_c
HHDDI 2-hydroxyhepta-2,4-diene-1,7-dioate isomerase 2hh24dd_c <=> 2ohed_c


CQW49_12520
<br>not added, not directly related to cellular metabolism --> https://www.genome.jp/dbget-bin/www_bget?ec:2.1.1.37 

In [954]:
for reaction in model_OB3b_old.genes.CQW49_12520.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

DMT DNA (cytosine-5-)-methyltransferase ahcys_c + dna5mtc_c + h_c --> amet_c + dnac_c


CQW49_12845

In [955]:
for reaction in model_OB3b_old.genes.CQW49_12845.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

H2CO3D H2CO3D co2_c + h2o_c <=> h2co3_c


In [956]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("carbonic", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

HCO3E carbonic anhydrase


In [958]:
iMsOB3b.reactions.HCO3E.gene_reaction_rule = 'CQW49_10875 or CQW49_20715 or CQW49_12845'

CQW49_12940

In [959]:
for reaction in model_OB3b_old.genes.CQW49_12940.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

HMGL HMGL hmgcoa_c --> acac_c + accoa_c


In [961]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("glutar", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

AKGDH 2 oxoglutarate dehydrogenase
AKGtex a ketoglutarate transport via diffusion  extracellular to periplasm 
EX_akg_e exchange flux   2 oxoglutarate
AKGt2rpp a ketoglutarate MFS transporter


In [963]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.HMGL)

Ignoring reaction 'HMGL' since it already exists.


CQW49_12970

In [965]:
for reaction in model_OB3b_old.genes.CQW49_12970.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

TRE6PP Trehalose-phosphatase h2o_c + tre6p_c --> pi_c + tre_c


In [966]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("trehal", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

TRE6PP trehalose 6 phosphate phosphatase
TRE6PS trehalose 6 phosphate synthase


In [968]:
iMsOB3b.reactions.TRE6PP.gene_reaction_rule = 'CQW49_03340 or CQW49_12970'

CQW49_13000 and CQW49_13005

In [970]:
for reaction in model_OB3b_old.genes.CQW49_13000.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

CYTAA3ipp cytochrome aa3 terminal oxidase - Complex IV 4.0 focytcc553_p + 8.0 h_c + o2_c --> 4.0 ficytcc553_p + 2.0 h2o_c + 4.0 h_p


In [972]:
for reaction in model_OB3b_old.genes.CQW49_13005.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

CYTAA3ipp cytochrome aa3 terminal oxidase - Complex IV 4.0 focytcc553_p + 8.0 h_c + o2_c --> 4.0 ficytcc553_p + 2.0 h2o_c + 4.0 h_p


In [971]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("cytochrome", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

CYO_2p cytochrome c oxidase
CYOO_4p cytochrome o ubiquinol oxidase
LDH_D3 D lactate dehydrogenase  cytochrome 
UQCYOR_2p ubiquinol cytochrome c reductase
CYOD cytochrome d terminal oxidase
NITR_NOpp Nitrite reductase (cytochrome; NO-forming)
NITR_AMpp Nitrite reductase (cytochrome; ammonia-forming)
CYP460ipp Cytochrome P460 - Hydroxylamine Dehydrogenase
NORZ2pp Nitric oxide reductase (cytochrome c)


In [974]:
iMsOB3b.reactions.CYO_2p.gene_reaction_rule = '((CQW49_10040 and CQW49_10020) or CQW49_13005) and (CQW49_10045 or CQW49_13000)'

CQW49_13305 
<br> unclear metabolites --> https://www.genome.jp/dbget-bin/www_bget?mtw:CQW49_13305

CQW49_13350 and CQW49_13355 
<br>this reaction will be added from old AM1 model

In [975]:
for reaction in model_OB3b_old.genes.CQW49_13350.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

GCCa Glycine-cleavage complex gly_c + h_c + lpro_c --> alpro_c + co2_c


In [976]:
for reaction in model_OB3b_old.genes.CQW49_13355.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

GCCa Glycine-cleavage complex gly_c + h_c + lpro_c --> alpro_c + co2_c


In [980]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("glycin", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

DALAOX glycine oxidase 2
PRFGS phosphoribosylformylglycinamide synthetase
GLYAMDTRc glycine amidinotransferase
GLYTRS glycine tRNA synthetase
GLYCOX3 glycine oxidase 3
GLYCOX4 glycine oxidase 4
GLYCOX5 glycine oxidase 5
GARFT phosphoribosylglycinamide formyltransferase
PRAGSr phosphoribosylglycinamide synthetase
GLYO1 glycine oxidase 1


CQW49_13470

In [982]:
for reaction in model_OB3b_old.genes.CQW49_13470.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

PRMICI 1-(5-phosphoribosyl)-5-[(5-phosphoribosylamino)methylideneamino)imidazole-4-carboxamide isomerase prfp_c --> prlp_c


In [983]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("formi", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

PRMICI N  5  phospho L ribosyl formimino  5 amino 1  5  phosphoribosyl  4 imidazolecarboxamide isomerase
NITR_NOpp Nitrite reductase (cytochrome; NO-forming)
NITR_AMpp Nitrite reductase (cytochrome; ammonia-forming)


In [986]:
iMsOB3b.reactions.PRMICI.gene_reaction_rule = 'CQW49_07615 or CQW49_13470'

CQW49_14985
<br> https://en.wikipedia.org/wiki/Hydroxylamine_reductase
<br> https://www.genome.jp/dbget-bin/www_bget?ec:1.7.99.1

In [987]:
for reaction in model_OB3b_old.genes.CQW49_14985.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

N2OR Nitrous oxide reductase (nadh) h_c + n2o_c + nadh_c --> h2o_c + n2_c + nad_c


In [989]:
iMsOB3b.reactions.PAMMOipp.gene_reaction_rule = 'CQW49_14985'

CQW49_15360

In [990]:
for reaction in model_OB3b_old.genes.CQW49_15360.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

HEX7 Hexokinase (D-fructose:ATP) atp_c + fru_c --> adp_c + f6p_c + h_c


In [991]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("Hexokinase", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

In [997]:
for reaction in iMsOB3b.metabolites.f6p_c.reactions:
    print(reaction)

FBP: fdp_c + h2o_c --> f6p_c + pi_c
MAN6PI: f6p_c <=> man6p_c
F6PPKL: f6p_c + pi_c <=> actp_c + e4p_c + h2o_c
PGI: f6p_c <=> g6p_c
GF6PTAr: f6p_c + gln_L_c --> gam6p_c + glu_L_c
TALA: g3p_c + s7p_c <=> e4p_c + f6p_c


In [998]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.HEX7)

CQW49_15505

In [999]:
for reaction in model_OB3b_old.genes.CQW49_15505.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

LIPOCT Lipoyl(octanoyl) transferase h_c + ocACP_c --> ACP_c + octapb_c


In [1006]:
model_OB3b_old.metabolites.ACP_c.name

'holo-[acyl-carrier protein]'

In [1007]:
model_OB3b_old.metabolites.ocACP_c.name

'octanoyl-[acyl-carrier protein]'

In [1008]:
model_OB3b_old.metabolites.octapb_c.name

'Octanoate (protein bound)'

In [1001]:
import re
for metabolite in iMsOB3b.metabolites:
    if re.search("octanoyl", metabolite.name, re.IGNORECASE):
        print(metabolite.id, metabolite.name)

3hoctACP_c 3R__3_hydroxyoctanoyl__acyl_carrier_protein_
3oocoa_c 3_oxooctanoyl_CoA
ocACP_c octanoyl__acyl_carrier_protein_
occoa_c octanoyl_CoA
3hocoa_c 3S__3_hydroxyoctanoyl_CoA
3ooctACP_c 3_oxooctanoyl__acyl_carrier_protein_
R_3hocoa_c 3R__3_hydroxyoctanoyl_CoA


In [1004]:
import re
for metabolite in iMsOB3b.metabolites:
    if re.search("acyl_carri", metabolite.name, re.IGNORECASE):
        print(metabolite.id, metabolite.name)

thex2eACP_c trans_hex_2_enoyl__acyl_carrier_protein_
tdeACP_c cis_tetradec_7_enoyl__acyl_carrier_protein_
toct2eACP_c trans_oct_2_enoyl__acyl_carrier_protein_
t3c9palmeACP_c trans_3_cis_9_hexadecenoyl__acyl_carrier_protein_
3opalmACP_c 3_oxohexadecanoyl__acyl_carrier_protein_
butACP_c butyryl__acyl_carrier_protein_
3ocpalm9eACP_c 3_oxo_cis_hexadec_9_enoyl__acyl_carrier_protein_
toctd2eACP_c trans_octadec_2_enoyl__acyl_carrier_protein_
apoACP_c apo__acyl_carrier_protein_
3ooctdACP_c 3_oxooctadecanoyl__acyl_carrier_protein_
myrsACP_c tetradecanoyl__acyl_carrier_protein_
3oddecACP_c 3_oxododecanoyl__acyl_carrier_protein_
R_3hbacp_c 3R__3_hydroxybutanoyl__acyl_carrier_protein_
t3c5ddeceACP_c trans_3_cis_5_dodecenoyl__acyl_carrier_protein_
3hcvac11eACP_c 3R__3_hydroxy_cis_octadec_11_enoyl__acyl_carrier_protein_
cddec5eACP_c cis_dodec_5_enoyl__acyl_carrier_protein_
3ohexACP_c 3_oxohexanoyl__acyl_carrier_protein_
maldcapoacp_c malonate_decarboxylase_apo__acyl_carrier_protein_
3hoctACP_c 3R__3

In [1010]:
iMsOB3b.metabolites.octapb_c

Metabolite identifier,octapb_c
Name,Octanoate (protein bound)
Memory address,0x07f328f3aab00
Formula,C8H15O
Compartment,c
In 2 reaction(s),"LIPOCT, LIPOS"


In [1011]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.LIPOCT)

CQW49_16115
<br> https://www.genome.jp/dbget-bin/www_bget?mtw:CQW49_16115

In [1012]:
for reaction in model_OB3b_old.genes.CQW49_16115.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

PFK_ppi Phosphofructokinase (ppi-dependent f6p_c + ppi_c <=> fdp_c + h_c + pi_c
PFK_2 Phosphofructokinase atp_c + tag6p__D_c --> adp_c + h_c + tagdp__D_c
PFK_3 Phosphofructokinase (s7p) atp_c + s7p_c --> adp_c + h_c + s17bp_c
PFK_3_ppi Pyrophosphate-dependent Phosphofructokinase (s7p) ppi_c + s7p_c <=> h_c + pi_c + s17bp_c


In [1013]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.PFK_ppi)

In [1014]:
from cobra import Model, Reaction, Metabolite
# Best practise: SBML compliant IDs

reaction = Reaction('PFK')
reaction.name = 'Phosphofructokinase'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default

iMsOB3b.add_reaction(reaction)
iMsOB3b.reactions.PFK.reaction = 'atp_c + f6p_c <=> adp_c + fdp_c + h_c'

CQW49_16125

In [1016]:
iMsOB3b.reactions.FDHr.gene_reaction_rule = 'CQW49_16615 and CQW49_16630 and (CQW49_16625 or CQW49_16125) and CQW49_16635'

CQW49_16180

In [1017]:
for reaction in model_OB3b_old.genes.CQW49_16180.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

ORPT Orotate phosphoribosyltransferase orot_c + prpp_c --> orot5p_c + ppi_c


In [1018]:
import re
for metabolite in iMsOB3b.metabolites:
    if re.search("Orotate", metabolite.name, re.IGNORECASE):
        print(metabolite.id, metabolite.name)

orot_c orotate
dhor_S_c dihydroorotate


In [1023]:
iMsOB3b.reactions.ORPT.gene_reaction_rule = 'CQW49_16180'

CQW49_16905

In [1024]:
for reaction in model_OB3b_old.genes.CQW49_16905.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

PGM_1 Phosphoglycerate mutase 3pg_c <=> 2pg_c


In [1026]:
iMsOB3b.reactions.PGM.gene_reaction_rule = 'CQW49_16905'

CQW49_17075

In [1027]:
for reaction in model_OB3b_old.genes.CQW49_17075.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

L_LACD3 L-Lactate dehydrogenase (menaquinone) lac__L_c + mqn8_c --> mql8_c + pyr_c
L_LACD2 L-Lactate dehydrogenase (ubiquinone) lac__L_c + q8_c --> pyr_c + q8h2_c


In [1029]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("lactate", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

L_LACt2rpp  S  lactate transport  periplasm to cytoplasm 
EX_lac_L_e exchange flux    S  lactate
LDH_L S lactate dehydrogenase
KARA1 2 acetolactate reductoisomerase
ACLS acetolactate synthase
L_LACtex  S  lactate transport via diffusion  extracellular to periplasm 
LDH_D3 D lactate dehydrogenase  cytochrome 


In [1031]:
iMsOB3b.reactions.LDH_D3.gene_reaction_rule = 'CQW49_17075 or CQW49_10060'

CQW49_17110

In [1032]:
for reaction in model_OB3b_old.genes.CQW49_17110.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

SQLC2 Squalene mutase (cyclizing) h2o_c + sql_c --> dpterol_c
SQLC Squalene mutase (cyclizing) sql_c --> dplpt_c


In [1034]:
model_OB3b_old.metabolites.sql_c

Metabolite identifier,sql_c
Name,squalene
Memory address,0x07f328f35cef0
Formula,C30H50
Compartment,c
In 3 reaction(s),"SQLC2, SQLC, BIOMASS_Mcapsulatus"


In [1035]:
model_OB3b_old.metabolites.dpterol_c

Metabolite identifier,dpterol_c
Name,hopan-22-ol
Memory address,0x07f328f2667f0
Formula,C30H52O
Compartment,c
In 2 reaction(s),"SQLC2, BIOMASS_Mcapsulatus"


In [1038]:
model_OB3b_old.metabolites.dplpt_c

Metabolite identifier,dplpt_c
Name,hop-22(29)-ene
Memory address,0x07f328f2667b8
Formula,C30H50
Compartment,c
In 1 reaction(s),SQLC


In [1039]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.SQLC)
iMsOB3b.add_reaction(model_OB3b_old.reactions.SQLC2)

Checkpoint: save gem now:

In [1041]:
cobra.io.save_json_model(iMsOB3b, memote_directory + draft_gems_dir_8 + "bigg_OB3b_15_01_2020.json")

CQW49_17220

In [7]:
for reaction in model_OB3b_old.genes.CQW49_17220.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

ACM6PH N-acetylmuramate 6-phosphate hydrolase acmum6p_c + h2o_c --> acgam6p_c + lac__D_c


In [8]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.ACM6PH)

CQW49_17240

In [9]:
for reaction in model_OB3b_old.genes.CQW49_17240.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

CHOLD Choline dehydrogenase chol_c + nad_c --> betald_c + h_c + nadh_c


In [10]:
model_OB3b_old.metabolites.betald_c

Metabolite identifier,betald_c
Name,Betaine aldehyde
Memory address,0x07fb215c04d68
Formula,C5H12NO
Compartment,c
In 1 reaction(s),CHOLD


In [11]:
iMsOB3b.metabolites.chol_c

Metabolite identifier,chol_c
Name,choline
Memory address,0x07fb210e7edd8
Formula,None
Compartment,c
In 1 reaction(s),GPDDA1


In [12]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.CHOLD)

CQW49_17595
<br>Gene annotation does not match up with the KEGG definition or genbank annotation --> https://www.genome.jp/dbget-bin/www_bget?mtw:CQW49_17595

In [13]:
for reaction in model_OB3b_old.genes.CQW49_17595.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

FBP Fructose-bisphosphatase fdp_c + h2o_c --> f6p_c + pi_c


CQW49_17895

In [14]:
for reaction in model_OB3b_old.genes.CQW49_17895.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

ORNTA_1 Ornithine transaminase akg_c + orn__L_c --> glu5sa_c + glu__L_c


In [15]:
iMsOB3b.reactions.ORNTA.gene_reaction_rule = 'CQW49_02635 or CQW49_17895'

CQW49_17905
<br>unclear definition, I am not sure which aldehyde dehydrogenase it is.
<br> https://www.genome.jp/dbget-bin/www_bget?ec:1.2.1.3

In [16]:
for reaction in model_OB3b_old.genes.CQW49_17905.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

AASAD3 AASAD3 L2aadp6sa_c + h2o_c + nad_c --> L2aadp_c + 2.0 h_c + nadh_c
ALDD2x Aldehyde dehydrogenase (acetaldehyde, NAD) acald_c + h2o_c + nad_c --> ac_c + 2.0 h_c + nadh_c


CQW49_18435

In [17]:
for reaction in model_OB3b_old.genes.CQW49_18435.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

TMDS3 Thymidylate synthase (FAD dependent) dump_c + h_c + mlthf_c + nadph_c --> dtmp_c + nadp_c + thf_c


In [18]:
print(iMsOB3b.metabolites.dump_c.reactions)
print(iMsOB3b.metabolites.mlthf_c.reactions)
print(iMsOB3b.metabolites.dtmp_c.reactions)
print(iMsOB3b.metabolites.thf_c.reactions)

frozenset({<Reaction DUTPDP at 0x7fb207cfada0>, <Reaction TMDS at 0x7fb207c643c8>})
frozenset({<Reaction GLYCL_3 at 0x7fb207492518>, <Reaction MOHMT at 0x7fb207ec4a20>, <Reaction FALDA at 0x7fb207d24710>, <Reaction MTHFR3 at 0x7fb20834df28>, <Reaction MTHFD at 0x7fb2073c0780>, <Reaction GHMT2r at 0x7fb2081aaac8>, <Reaction TMDS at 0x7fb207c643c8>})
frozenset({<Reaction DTMPK at 0x7fb207347080>, <Reaction TMDS at 0x7fb207c643c8>, <Reaction NTPP7 at 0x7fb20823d470>})
frozenset({<Reaction MOHMT at 0x7fb207ec4a20>, <Reaction GARFT at 0x7fb20818d630>, <Reaction AICART at 0x7fb207827470>, <Reaction DHFR at 0x7fb207ce1080>, <Reaction GHMT2r at 0x7fb2081aaac8>, <Reaction METSr at 0x7fb207fe34e0>, <Reaction FTHFL at 0x7fb207c88ef0>, <Reaction Biomass_Mextorquens_AM1_core at 0x7fb2083fdb00>, <Reaction GLYCL_3 at 0x7fb207492518>, <Reaction FALDA at 0x7fb207d24710>, <Reaction FMETTRS at 0x7fb207cfa7f0>})


In [19]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.TMDS3)

CQW49_18575
<br> Gene codes for tRNA dimethylallyltransferase [EC:2.5.1.75] which cannot be included into GEM
<br> https://www.genome.jp/dbget-bin/www_bget?mtw:CQW49_18575

In [20]:
for reaction in model_OB3b_old.genes.CQW49_18575.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

CQW49_18780

In [21]:
for reaction in model_OB3b_old.genes.CQW49_18780.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

NTPP11 Nucleoside triphosphate pyrophosphorylase (xtp) h_c + ppi_c + xmp_c --> h2o_c + xtp_c
NTPP8 Nucleoside triphosphate pyrophosphorylase (utp) h_c + ppi_c + ump_c --> h2o_c + utp_c
NTPP10 Nucleoside triphosphate pyrophosphorylase (ditp) ditp_c + h2o_c --> dimp_c + h_c + ppi_c
DUTPDP DUTP diphosphatase dutp_c + h2o_c --> dump_c + h_c + ppi_c
NTPP2 Nucleoside triphosphate pyrophosphorylase (gtp) gtp_c + h2o_c --> gmp_c + h_c + ppi_c
NTPP9 Nucleoside triphosphate pyrophosphorylase (itp) h2o_c + itp_c --> h_c + imp_c + ppi_c
NTPP1 Nucleoside triphosphate pyrophosphorylase (dgtp) dgtp_c + h2o_c --> dgmp_c + h_c + ppi_c


In [22]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("nucleoside triphosphate", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

NTPP7 deoxynucleoside triphosphate pyrophosphohydrolase  dTTP 
NTPP8 nucleoside triphosphate pyrophosphohydrolase  UTP 
NTPP3 deoxynucleoside triphosphate pyrophosphohydrolase  dCTP 
NTPP4 nucleoside triphosphate pyrophosphohydrolase  CTP 
NTPP5 deoxynucleoside triphosphate pyrophosphohydrolase  dATP 
NTPP6 nucleoside triphosphate pyrophosphohydrolase  ATP 
NTPP1 deoxynucleoside triphosphate pyrophosphohydrolase  dGTP 
NTPP2 nucleoside triphosphate pyrophosphohydrolase  GTP 
NTPTP1 Nucleoside triphosphate tripolyhydrolase


In [23]:
iMsOB3b.reactions.NTPP8.gene_reaction_rule = 'CQW49_01615 or CQW49_18780'
iMsOB3b.reactions.NTPP6.gene_reaction_rule = 'CQW49_01615 or CQW49_11120'
iMsOB3b.reactions.NTPP4.gene_reaction_rule = 'CQW49_01615 or CQW49_11120'
iMsOB3b.reactions.NTPP2.gene_reaction_rule = 'CQW49_01615 or CQW49_11120'

CQW49_19460
<br> Not clarified enough
<br> https://www.genome.jp/dbget-bin/www_bget?ec:3.4.11.2

In [24]:
for reaction in model_OB3b_old.genes.CQW49_19460.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

AMPTASEPG Aminopeptidase (pro-gly) h2o_c + progly_c --> gly_c + pro__L_c
AMPTASECG Alanyl aminopeptidase (cys-gly) cgly_c + h2o_c --> cys__L_c + gly_c


CQW49_20020

In [25]:
for reaction in model_OB3b_old.genes.CQW49_20020.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

OXADC Oxalate decarboxylase h_c + oxa_c --> co2_c + for_c


In [26]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.OXADC)

CQW49_20165
<br> https://www.genome.jp/dbget-bin/www_bget?ec:2.10.1.1

In [27]:
for reaction in model_OB3b_old.genes.CQW49_20165.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

BWCOS Tungsten bispterin cofactor synthase mptamp_c + wco_c --> amp_c + bwco_c + cu2_c
BMOCOS Bis-molybdenum cofactor synthase moco_c + mptamp_c --> amp_c + bmoco_c + cu2_c
MOCOS Molybdenum cofactor synthase 2.0 h_c + mobd_c + mptamp_c --> amp_c + cu2_c + h2o_c + moco_c
WCOS Tungsten pterin cofactor synthase 2.0 h_c + mptamp_c + tungs_c --> amp_c + cu2_c + h2o_c + wco_c


In [28]:
model_OB3b_old.reactions.BMOCOS

Reaction identifier,BMOCOS
Name,Bis-molybdenum cofactor synthase
Memory address,0x07fb2156ddda0
Stoichiometry,moco_c + mptamp_c --> amp_c + bmoco_c + cu2_c Molybdenum cofactor + Adenylated molybdopterin --> AMP + Bis-molybdenum cofactor + Copper
GPR,CQW49_20165
Lower bound,0.0
Upper bound,1000.0


In [29]:
model_OB3b_old.reactions.BWCOS

Reaction identifier,BWCOS
Name,Tungsten bispterin cofactor synthase
Memory address,0x07fb2156ddf28
Stoichiometry,mptamp_c + wco_c --> amp_c + bwco_c + cu2_c Adenylated molybdopterin + Tungsten binding cofactor --> AMP + Tungsten bispterin cofactor + Copper
GPR,CQW49_20165
Lower bound,0.0
Upper bound,1000.0


In [30]:
model_OB3b_old.reactions.WCOS

Reaction identifier,WCOS
Name,Tungsten pterin cofactor synthase
Memory address,0x07fb216d60be0
Stoichiometry,2.0 h_c + mptamp_c + tungs_c --> amp_c + cu2_c + h2o_c + wco_c 2.0 H(+) + Adenylated molybdopterin + Tungstate --> AMP + Copper + H2O + Tungsten binding cofactor
GPR,CQW49_20165
Lower bound,0.0
Upper bound,1000.0


In [31]:
model_OB3b_old.reactions.MOCOS

Reaction identifier,MOCOS
Name,Molybdenum cofactor synthase
Memory address,0x07fb21558c1d0
Stoichiometry,2.0 h_c + mobd_c + mptamp_c --> amp_c + cu2_c + h2o_c + moco_c 2.0 H(+) + Molybdate + Adenylated molybdopterin --> AMP + Copper + H2O + Molybdenum cofactor
GPR,CQW49_20165
Lower bound,0.0
Upper bound,1000.0


In [32]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("molybdo", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

MOBDMPTL molybdenum  molybdopterin ligase
MOGDS molybdopterin guanine dinucleotide synthase
MPTAT molybdopterin adenylyltransferase
MPTSA molybdopterin synthase activations
MPTSS molybdopterin synthase sulfurylase


In [33]:
iMsOB3b.reactions.MOBDMPTL.gene_reaction_rule = 'CQW49_20165'

CQW49_20215

In [34]:
for reaction in model_OB3b_old.genes.CQW49_20215.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

In [35]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("carbonic", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

HCO3E carbonic anhydrase


In [36]:
iMsOB3b.reactions.HCO3E.gene_reaction_rule = 'CQW49_10875 or CQW49_20715 or CQW49_12845 or CQW49_20215'

CQW49_20655
<br> different annotation between kegg, genbank and OB3b-related gem https://www.genome.jp/dbget-bin/www_bget?mtw:CQW49_20655

In [37]:
for reaction in model_OB3b_old.genes.CQW49_20655.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

2OH3K5MPPISO 2-hydroxy-3-keto-5-methylthiopentenyl-1-phosphate phosphatase 2h3kmtp_c + h2o_c --> 12dmpo_c + h_c + pi_c


In [38]:
print(iMsOB3b.metabolites.get_by_id('2h3kmpt_c'))

KeyError: '2h3kmpt_c'

In [40]:
print(iMsOB3b.metabolites.get_by_id('12dmpo_c'))

KeyError: '12dmpo_c'

CQW49_20795
<br> https://www.genome.jp/dbget-bin/www_bget?mtw:CQW49_20795

In [41]:
for reaction in model_OB3b_old.genes.CQW49_20795.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

ACS2 ACS2 atp_c + coa_c + ppa_c --> amp_c + ppcoa_c + ppi_c


In [42]:
iMsOB3b.reactions.ACS.gene_reaction_rule = 'CQW49_20795 or CQW49_13835'

CQW49_21030

In [43]:
for reaction in model_OB3b_old.genes.CQW49_21030.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

METGL METGL h2o_c + met__L_c --> 2obut_c + ch4s_c + nh4_c


In [44]:
model_OB3b_old.reactions.METGL

Reaction identifier,METGL
Name,METGL
Memory address,0x07fb2152c6668
Stoichiometry,h2o_c + met__L_c --> 2obut_c + ch4s_c + nh4_c H2O + L-methionine zwitterion --> 2-oxobutanoate + Methanethiol CH4S + Ammonium
GPR,CQW49_21030
Lower bound,0.0
Upper bound,1000.0


In [45]:
iMsOB3b.metabolites.met__L_c

Metabolite identifier,met__L_c
Name,L-methionine zwitterion
Memory address,0x07fb20f5f0e48
Formula,C5H11NO2S
Compartment,c
In 3 reaction(s),"CPPPGO2, LIPOS, METSOXR2"


In [46]:
iMsOB3b.metabolites.get_by_id('2obut_c')

Metabolite identifier,2obut_c
Name,2_oxobutanoate
Memory address,0x07fb20fd93470
Formula,None
Compartment,c
In 2 reaction(s),"THRD_L, ACHBS"


In [47]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.METGL)

CQW49_21250 -->
<br> too broad definition --> https://www.genome.jp/dbget-bin/www_bget?ec:4.2.1.17

In [48]:
for reaction in model_OB3b_old.genes.CQW49_21250.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

ECOAH2 3-hydroxyacyl-CoA dehydratase (3-hydroxyhexanoyl-CoA) 3hhcoa_c <=> h2o_c + hx2coa_c
ECOAH7 3-hydroxyacyl-CoA dehydratase (3-hydroxyhexadecanoyl-CoA) 3hhdcoa_c <=> h2o_c + hdd2coa_c
ECOAH3 3-hydroxyacyl-CoA dehydratase (3-hydroxyoctanoyl-CoA) 3hocoa_c <=> h2o_c + oc2coa_c
DHACOAH 2,3-dehydroadipyl-CoA hydratase 23dhacoa_c + h2o_c <=> 3hadpcoa_c
ECOAH6 3-hydroxyacyl-CoA dehydratase (3-hydroxytetradecanoyl-CoA) 3htdcoa_c <=> h2o_c + td2coa_c
ECOAH5 3-hydroxyacyl-CoA dehydratase (3-hydroxydodecanoyl-CoA) 3hddcoa_c <=> dd2coa_c + h2o_c
ECOAH4 3-hydroxyacyl-CoA dehydratase (3-hydroxydecanoyl-CoA) 3hdcoa_c <=> dc2coa_c + h2o_c


CQW49_21325 and CQW49_21330 and CQW49_21335

In [49]:
for reaction in model_OB3b_old.genes.CQW49_21325.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

UREA Urease h2o_c + 2.0 h_c + urea_c --> co2_c + 2.0 nh4_c


In [50]:
for reaction in model_OB3b_old.genes.CQW49_21330.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

UREA Urease h2o_c + 2.0 h_c + urea_c --> co2_c + 2.0 nh4_c


In [51]:
for reaction in model_OB3b_old.genes.CQW49_21335.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

UREA Urease h2o_c + 2.0 h_c + urea_c --> co2_c + 2.0 nh4_c


In [52]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("urea", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

In [53]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.UREA)

In [54]:
iMsOB3b.reactions.UREA.gene_reaction_rule = 'CQW49_21325 and CQW49_21330 and CQW49_21335'

CQW49_21545
<br> unclear enzyme function --> https://www.genome.jp/dbget-bin/www_bget?mtw:CQW49_21545

In [55]:
for reaction in model_OB3b_old.genes.CQW49_21545.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

ACPS1 Acyl-carrier protein synthase apoACP_c + coa_c --> ACP_c + h_c + pap_c


CQW49_21585

In [56]:
for reaction in model_OB3b_old.genes.CQW49_21585.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

NICRNS NICRNS atp_c + nicrns_c --> adp_c + h_c + nicrnt_c
NMNHYD Nicotinamide ribonucleotide phosphohydrolase h2o_c + nmn_c --> pi_c + rnam_c


CQW49_21985

In [57]:
for reaction in model_OB3b_old.genes.CQW49_21985.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

SCYSDS SUF Cysteine desulfuration cys__L_c + sufse_c --> ala__L_c + sufsesh_c
S2FE2SR SUF [2Fe-2S] regeneration 2fe1s_c + atp_c + h2o_c + sufbcd_c + sufsesh_c --> adp_c + 5.0 h_c + pi_c + sufbcd_2fe2s_c + sufse_c


In [58]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.SCYSDS)

CQW49_22035

In [59]:
for reaction in model_OB3b_old.genes.CQW49_22035.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

MOCDS Molybdopterin cytidine dinucleotide synthase ctp_c + h_c + moco_c --> mococdp_c + ppi_c
XAND Xanthine dehydrogenase h2o_c + nad_c + xan_c --> h_c + nadh_c + urate_c
HXAND Hypoxanthine dehydrogenase h2o_c + hxan_c + nad_c --> h_c + nadh_c + xan_c


In [60]:
model_OB3b_old.reactions.MOCDS

Reaction identifier,MOCDS
Name,Molybdopterin cytidine dinucleotide synthase
Memory address,0x07fb215574e48
Stoichiometry,ctp_c + h_c + moco_c --> mococdp_c + ppi_c CTP(3-) + H(+) + Molybdenum cofactor --> Molybdopterin cytosine dinucleotide + diphosphate(3-)
GPR,CQW49_22035
Lower bound,0.0
Upper bound,1000.0


In [61]:
iMsOB3b.metabolites.moco_c

Metabolite identifier,moco_c
Name,molybdenum_cofactor
Memory address,0x07fb210479320
Formula,None
Compartment,c
In 2 reaction(s),"MOBDMPTL, MOGDS"


In [62]:
iMsOB3b.reactions.MOGDS

Reaction identifier,MOGDS
Name,molybdopterin guanine dinucleotide synthase
Memory address,0x07fb207a9b470
Stoichiometry,gtp_c + h_c + moco_c <=> mocogdp_c + ppi_c GTP + H_ + molybdenum_cofactor <=> molybdopterin_guanine_dinucleotide + diphosphate
GPR,META1_3049 or (CQW49_12160 and META1_0772)
Lower bound,-1000.0
Upper bound,1000.0


In [63]:
iMsOB3b.reactions.MOBDMPTL

Reaction identifier,MOBDMPTL
Name,molybdenum molybdopterin ligase
Memory address,0x07fb207603780
Stoichiometry,h_c + mobd_c + mptamp_c --> amp_c + h2o_c + moco_c H_ + molybdate + molybdopterin_AMP --> AMP + H2O + molybdenum_cofactor
GPR,CQW49_20165
Lower bound,0.0
Upper bound,1000.0


In [64]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.MOCDS)

CQW49_22875

In [65]:
for reaction in model_OB3b_old.genes.CQW49_22875.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

SUCDH_q8_pp succinate dehydrogenase q8_im + succ_c <=> fum_c + q8h2_im


In [66]:
iMsOB3b.reactions.SUCCD1

Reaction identifier,SUCCD1
Name,succinate dehydrogenase
Memory address,0x07fb207acbb38
Stoichiometry,q_c + succ_c <=> fum_c + qh2_c UQ + succinate <=> fumarate + UQH2
GPR,CQW49_22885 and CQW49_10810 and (CQW49_10825 or CQW49_22875) and CQW49_10800
Lower bound,-1000.0
Upper bound,1000.0


CQW49_22910

In [67]:
for reaction in model_OB3b_old.genes.CQW49_22910.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

MCITD 2-methylcitrate dehydratase 2mcit_c --> 2mcacn_c + h2o_c


In [68]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.MCITD)

CQW49_22935
<br> https://www.genome.jp/dbget-bin/www_bget?mtw:CQW49_22935

In [69]:
for reaction in model_OB3b_old.genes.CQW49_22935.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

In [70]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("nitri", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

NTRIR4 ferredoxin  nitrite reductase
NODOy nitric oxide dioxygenase
EX_no2_e nitrite exchange
NO2tex nitrite transport via diffusion (extracellular to periplasm)
NO3t7pp Nitrate transport in via nitrite antiport (periplasm)
NITR_NOpp Nitrite reductase (cytochrome; NO-forming)
NITR_AMpp Nitrite reductase (cytochrome; ammonia-forming)
NTRIR2x Nitrite Reductase (NADH)
NORZ2pp Nitric oxide reductase (cytochrome c)


In [71]:
iMsOB3b.reactions.NORZ2pp.gene_reaction_rule = 'CQW49_22935'

CQW49_23115
<br> too broad definition in the KEGG https://www.genome.jp/dbget-bin/www_bget?mtw:CQW49_23115

In [72]:
for reaction in model_OB3b_old.genes.CQW49_23115.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

In [73]:
import re
for reaction in iMsOB3b.reactions:
    if re.search("glutath", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name)

GTHS glutathione synthetase
LGTHL lactoylglutathione lyase
GTHPp glutathione peroxidase  periplasmic 
GTHOr glutathione reductase
GLYOX S lactoylglutathione hydrolase
GTHRDHpp Glutathione hydralase (periplasmic)


CQW49_23740
<br> too broad definition

In [74]:
for reaction in model_OB3b_old.genes.CQW49_23740.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

KAS2 B-ketoacyl synthetase (n-C14:0) accoa_c + 17.0 h_c + 6.0 malcoa_c + 12.0 nadph_c --> 6.0 co2_c + 7.0 coa_c + 5.0 h2o_c + 12.0 nadp_c + ttdca_c
KAS13 B-ketoacyl synthetase (octadecanoate) accoa_c + 23.0 h_c + 8.0 malcoa_c + 16.0 nadph_c --> 8.0 co2_c + 9.0 coa_c + 7.0 h2o_c + 16.0 nadp_c + ocdca_c
KAS16 3-hydroxy-myristoyl-ACP synthesis ddcaACP_c + 2.0 h_c + malACP_c + nadph_c --> 3hmrsACP_c + ACP_c + co2_c + nadp_c
KAS8 B-ketoacyl synthetase (palmitate, n-C16:0) accoa_c + 20.0 h_c + 7.0 malcoa_c + 14.0 nadph_c --> 7.0 co2_c + 8.0 coa_c + 6.0 h2o_c + hdca_c + 14.0 nadp_c
KAS7 B-ketoacyl synthetase (n-C16:1) accoa_c + 19.0 h_c + 7.0 malcoa_c + 13.0 nadph_c --> 7.0 co2_c + 8.0 coa_c + 6.0 h2o_c + hdcea_c + 13.0 nadp_c
KAS17 B-ketoacyl synthetase (n-C18:1) accoa_c + 22.0 h_c + 8.0 malcoa_c + 15.0 nadph_c --> 8.0 co2_c + 9.0 coa_c + 7.0 h2o_c + 15.0 nadp_c + ocdcea_c


CQW49_23910

In [76]:
for reaction in model_OB3b_old.genes.CQW49_23910.reactions:
    print(reaction.id, reaction.name, reaction.reaction)

NMNS NMN synthetase h_c + ncam_c + prpp_c --> nmn_c + ppi_c


In [77]:
model_OB3b_old.reactions.NMNS

Reaction identifier,NMNS
Name,NMN synthetase
Memory address,0x07fb21527ef28
Stoichiometry,h_c + ncam_c + prpp_c --> nmn_c + ppi_c H(+) + nicotinamide + 5-phosphoribosyl diphosphate --> beta-nicotinamide D-ribonucleotide + diphosphate(3-)
GPR,CQW49_23910
Lower bound,0.0
Upper bound,1000.0


In [78]:
iMsOB3b.add_reaction(model_OB3b_old.reactions.NMNS)

Save model again:

In [79]:
cobra.io.save_json_model(iMsOB3b, memote_directory + draft_gems_dir_8 + "bigg_OB3b_16_01_2020.json")

CQW49_00915

In [89]:
import re
for reaction in model_AM1.reactions:
    if re.search("molyb", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name, reaction.reaction)

MOBDMPTL molybdenum__molybdopterin_ligase h_c + mobd_c + mptamp_c --> amp_c + h2o_c + moco_c
MOBDabcpp molybdate_ABC_transporter atp_c + h2o_c + mobd_p --> adp_c + h_c + mobd_c + pi_c
MOGDS molybdopterin_guanine_dinucleotide_synthase gtp_c + h_c + moco_c <=> mocogdp_c + ppi_c
EX_mobd_e_ exchange_flux___molybdate mobd_e <=> 
MOBDtex molybdate_transport_via_diffusion__extracellular_to_periplasm_ mobd_e <=> mobd_p
MPTAT molybdopterin_adenylyltransferase atp_c + h_c + mpt_c --> mptamp_c + ppi_c
MPTSA molybdopterin_synthase_activations Ssact_c + mptS_c --> 2.0 h_c + mptSs_c + uSsact_c
MPTSS molybdopterin_synthase_sulfurylase 2.0 mptSs_c + precZ_c --> 2.0 mptS_c + mpt_c
MPTSP molybderin_biosynthesis_protein gtp_c + h2o_c --> 2.0 h_c + ppi_c + precZ_c


In [93]:
iMsOB3b.reactions.MOGDS.gene_reaction_rule = 'CQW49_12160 or CQW49_00915'

CQW49_13350 and CQW49_13355

In [94]:
import re
for reaction in model_AM1.reactions:
    if re.search("glycin", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name, reaction.reaction)

DALAOX glycine_oxidase_2 ala_D_c + h2o_c + o2_c --> h2o2_c + nh4_c + pyr_c
PRFGS phosphoribosylformylglycinamide_synthetase atp_c + fgam_c + gln_L_c + h2o_c --> adp_c + fpram_c + glu_L_c + h_c + pi_c
GLYAMDTRc glycine_amidinotransferase arg_L_c + gly_c --> gudac_c + orn_c
GLYTRS glycine_tRNA_synthetase atp_c + gly_c + trnagly_c --> amp_c + glytrna_c + ppi_c
GCCa glycine_decarboxylase Gcvpll_c + gly_c --> co2_c + pSamdhll_c
GLYCOX3 glycine_oxidase_3 h2o_c + o2_c + sarcs_c --> glx_c + h2o2_c + mma_c
GLYCOX4 glycine_oxidase_4 etgly_c + h2o_c + o2_c --> ethlam_c + glx_c + h2o2_c
GLYCOX5 glycine_oxidase_5 gly_c + o2_c --> h2o2_c + imgly_c
GARFT phosphoribosylglycinamide_formyltransferase 10fthf_c + gar_c <=> fgam_c + h_c + thf_c
PRAGSr phosphoribosylglycinamide_synthetase atp_c + gly_c + pram_c <=> adp_c + gar_c + h_c + pi_c
GLYO1 glycine_oxidase_1 gly_c + h2o_c + o2_c --> glx_c + h2o2_c + nh4_c


In [97]:
model_AM1.reactions.GCCa.gene_reaction_rule = 'CQW49_13350 and CQW49_13355'

CQW49_01205

In [100]:
import re
for reaction in model_AM1.reactions:
    if re.search("di_haem", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name, reaction.reaction)

DHCYTCPEROX di_haem_cytochrome_c_peroxidase 2.0 focytc_c + h2o2_p + 2.0 h_c --> 2.0 ficytc_c + 2.0 h2o_p


In [102]:
iMsOB3b.add_reaction(model_AM1.reactions.DHCYTCPEROX)

In [106]:
iMsOB3b.reactions.DHCYTCPEROX.gene_reaction_rule = 'CQW49_01205'

CQW49_08960

In [107]:
import re
for reaction in model_AM1.reactions:
    if re.search("glutathione", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name, reaction.reaction)

GTHS glutathione_synthetase atp_c + glucys_c + gly_c <=> adp_c + gthrd_c + h_c + pi_c
LGTHL lactoylglutathione_lyase lgt_S_c <=> gthrd_c + mthgxl_c
GTHPp glutathione_peroxidase__periplasmic_ 2.0 gthrd_c + h2o2_p --> gthox_c + 2.0 h2o_p
GTHOr glutathione_reductase gthox_c + h_c + nadph_c --> 2.0 gthrd_c + nadp_c
GLYOX S_lactoylglutathione_hydrolase h2o_c + lgt_S_c <=> gthrd_c + h_c + lac_D_c


In [108]:
import re
for reaction in model_AM1.reactions:
    if re.search("deaminase", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name, reaction.reaction)

THRD_L threonine_deaminase thr_L_c --> 2obut_c + nh4_c
DHPPDA2 diaminohydroxyphosphoribosylaminopyrimidine_deaminase 25drapp_c + h2o_c + h_c --> 5apru_c + nh4_c
ADD adenine_deaminase ade_c + h2o_c + h_c --> hxan_c + nh4_c
DCYTD cytidine_deaminase_2 dcyt_c + h2o_c + h_c --> duri_c + nh4_c
ORNCD ornithine_cyclodeaminase orn_c --> nh4_c + pro_L_c
SERD_L L_serine_deaminase ser_L_c --> nh4_c + pyr_c
HMBS porphobilinogen_deaminase h2o_c + 4.0 ppbng_c <=> hmbil_c + 4.0 nh4_c
DCTPD dCTP_deaminase dctp_c + h2o_c + h_c --> dutp_c + nh4_c
CYTD cytidine_deaminase_1 cytd_c + h2o_c + h_c --> nh4_c + uri_c
GUAD guanine_deaminase gua_c + h2o_c + h_c <=> nh4_c + xan_c


CQW49_17395

In [110]:
import re
for reaction in model_AM1.reactions:
    if re.search("magnesium", reaction.name, re.IGNORECASE):
        print(reaction.id, reaction.name, reaction.reaction)

MPML magnesium_chelatase atp_c + h2o_c + mg2_c + ppp9_c --> adp_c + 3.0 h_c + mppp9_c + pi_c
EX_mg2_e_ exchange_flux___magnesium mg2_e <=> 
AMP9MC aerobic_magnesium_protoporphyrin_IX_monomethyl_ester_cyclase_1 nadph_c + o2_c + omgppp9me_c --> dvpclphd_c + 2.0 h2o_c + nadp_c
AMP9MC2 aerobic_magnesium_protoporphyrin_IX_monomethyl_ester_cyclase_2 h_c + hmgppp9me_c + nadph_c + o2_c --> 2.0 h2o_c + nadp_c + omgppp9me_c
AMP9MC3 aerobic_magnesium_protoporphyrin_IX_monomethyl_ester_cyclase_3 h_c + mgppp9me_c + nadph_c + o2_c --> h2o_c + hmgppp9me_c + nadp_c
MG2tex magnesium_transport_via_diffusion__extracellular_to_periplasm_ mg2_e <=> mg2_p
MP9MT magnesium_protoporphyrin_IX_methyltransferase amet_c + mppp9_c --> ahcys_c + mgppp9me_c
MG2uabcpp magnesium_ABC_transporter atp_c + h2o_c + mg2_p --> adp_c + h_c + mg2_c + pi_c


In [112]:
iMsOB3b.reactions.MG2uabcpp.gene_reaction_rule = 'CQW49_12745 or CQW49_17395'

Save model before proceeding further:

In [114]:
cobra.io.save_json_model(iMsOB3b, memote_directory + draft_gems_dir_8 + "bigg_OB3b_16_01_2020.json")